# Data analysis

#### Imports

In [2018]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt

#### Connect to the database

In [2019]:
database = "data/database.sqlite"
conn = sqlite3.connect(database)

### Merge data from **League**, **Country** and **Match**

In [2020]:
matches = pd.read_sql("""SELECT *
                        FROM League
                        JOIN Country ON Country.id = League.country_id
                        JOIN Match ON Match.country_id = League.country_id;""", conn)
# Drop some unuseful columns and small renaming
matches.drop("id", axis=1, inplace=True)
matches.drop("country_id", axis=1, inplace=True)
matches.columns.values[0] = "league_name"
matches.columns.values[1] = "country_name"
matches

,league_name,country_name,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,Belgium Jupiler League,Belgium,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
1,Belgium Jupiler League,Belgium,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
2,Belgium Jupiler League,Belgium,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
3,Belgium Jupiler League,Belgium,1,2008/2009,1,2008-08-16 00:00:00,492479,9999,8571,2,...,3.60,2.10,3.25,3.00,1.85,3.25,3.75,2.10,3.25,3.10
4,Belgium Jupiler League,Belgium,1,2008/2009,1,2008-08-16 00:00:00,492480,4049,9996,1,...,2.40,2.80,3.25,2.25,2.80,3.20,2.25,2.88,3.25,2.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,Switzerland Super League,Switzerland,24558,2015/2016,36,2016-05-25 00:00:00,1992225,9931,9956,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,Switzerland Super League,Switzerland,24558,2015/2016,36,2016-05-25 00:00:00,1992226,7896,10190,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,Switzerland Super League,Switzerland,24558,2015/2016,36,2016-05-25 00:00:00,1992227,10199,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,Switzerland Super League,Switzerland,24558,2015/2016,36,2016-05-25 00:00:00,1992228,10191,10192,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2021]:
matches.columns.values

array(['league_name', 'country_name', 'league_id', 'season', 'stage',
       'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1',
       'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2',
       'away_player_X3', 'away_player_X4', 'away_player_X5',
       'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6',
       'home_player_Y7', 'home_player_Y8', 'home_player_Y9',
       'home_player_Y10', 'home_player_Y11', 'away_player_Y1',
       'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7',

In [2022]:
# Find indices for columns that have no explanation and can be dropped
matches.columns.values[11:55]

array(['home_player_X1', 'home_player_X2', 'home_player_X3',
       'home_player_X4', 'home_player_X5', 'home_player_X6',
       'home_player_X7', 'home_player_X8', 'home_player_X9',
       'home_player_X10', 'home_player_X11', 'away_player_X1',
       'away_player_X2', 'away_player_X3', 'away_player_X4',
       'away_player_X5', 'away_player_X6', 'away_player_X7',
       'away_player_X8', 'away_player_X9', 'away_player_X10',
       'away_player_X11', 'home_player_Y1', 'home_player_Y2',
       'home_player_Y3', 'home_player_Y4', 'home_player_Y5',
       'home_player_Y6', 'home_player_Y7', 'home_player_Y8',
       'home_player_Y9', 'home_player_Y10', 'home_player_Y11',
       'away_player_Y1', 'away_player_Y2', 'away_player_Y3',
       'away_player_Y4', 'away_player_Y5', 'away_player_Y6',
       'away_player_Y7', 'away_player_Y8', 'away_player_Y9',
       'away_player_Y10', 'away_player_Y11'], dtype=object)

#### Drop columns with no explanation
We will drop columns fro home_player_X..., away_player_X..., home_player_Y... and away_player_Y... as they are unclear in meaning

In [2023]:
matches.drop(matches.columns[11:55], axis=1, inplace=True)

Now we are left with home_player_... and away_player_... that simply stores value of **player_api_id**, that can be used to retrieve player from **Player** table

In [2024]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 71 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   league_name       25979 non-null  object 
 1   country_name      25979 non-null  object 
 2   league_id         25979 non-null  int64  
 3   season            25979 non-null  object 
 4   stage             25979 non-null  int64  
 5   date              25979 non-null  object 
 6   match_api_id      25979 non-null  int64  
 7   home_team_api_id  25979 non-null  int64  
 8   away_team_api_id  25979 non-null  int64  
 9   home_team_goal    25979 non-null  int64  
 10  away_team_goal    25979 non-null  int64  
 11  home_player_1     24755 non-null  float64
 12  home_player_2     24664 non-null  float64
 13  home_player_3     24698 non-null  float64
 14  home_player_4     24656 non-null  float64
 15  home_player_5     24663 non-null  float64
 16  home_player_6     24654 non-null  float6

##### Clear data and prepare for ML

In [2025]:
matches['date']= pd.to_datetime(matches['date'])
matches.drop(['country_name', 'league_id'], axis=1, inplace=True) # These colums will not be needed, will one-hot encode 'league_name' instead
matches = matches.loc[:,~matches.columns.duplicated()].copy() # remove duplicate columns
matches = pd.get_dummies(data=matches, columns=['league_name'])
# Fill NaNs with mean values
#matches.fillna(value=np.nan, inplace=True)
#matches.fillna(matches.mean(), inplace=True)
matches

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,league_name_England Premier League,league_name_France Ligue 1,league_name_Germany 1. Bundesliga,league_name_Italy Serie A,league_name_Netherlands Eredivisie,league_name_Poland Ekstraklasa,league_name_Portugal Liga ZON Sagres,league_name_Scotland Premier League,league_name_Spain LIGA BBVA,league_name_Switzerland Super League
0,2008/2009,1,2008-08-16,492474,10000,9994,0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2008/2009,1,2008-08-16,492475,9984,8635,0,3,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2008/2009,1,2008-08-16,492477,7947,9985,1,3,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2008/2009,1,2008-08-16,492479,9999,8571,2,2,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,2008/2009,1,2008-08-16,492480,4049,9996,1,2,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,2015/2016,36,2016-05-25,1992225,9931,9956,0,1,42113.0,458806.0,...,0,0,0,0,0,0,0,0,0,1
25975,2015/2016,36,2016-05-25,1992226,7896,10190,3,0,330458.0,282287.0,...,0,0,0,0,0,0,0,0,0,1
25976,2015/2016,36,2016-05-25,1992227,10199,10179,2,2,42276.0,114792.0,...,0,0,0,0,0,0,0,0,0,1
25977,2015/2016,36,2016-05-25,1992228,10191,10192,0,3,462944.0,239959.0,...,0,0,0,0,0,0,0,0,0,1


In [2026]:
# Find indices of columns that 
matches.columns.values[8:30]

array(['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',
       'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',
       'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11'], dtype=object)

In [2027]:
# Watch out for NaNs as player_api_id !!!
# for column in matches.columns.values[8:30]:
#     matches[column] = matches[column].astype('int64')
# matches

In [2028]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 79 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   season                                25979 non-null  object        
 1   stage                                 25979 non-null  int64         
 2   date                                  25979 non-null  datetime64[ns]
 3   match_api_id                          25979 non-null  int64         
 4   home_team_api_id                      25979 non-null  int64         
 5   away_team_api_id                      25979 non-null  int64         
 6   home_team_goal                        25979 non-null  int64         
 7   away_team_goal                        25979 non-null  int64         
 8   home_player_1                         24755 non-null  float64       
 9   home_player_2                         24664 non-null  float64       
 10

In [2029]:
# Find indices of columns of odds for home win, draw and away win
matches.columns.values[29:69]

array(['away_player_11', 'goal', 'shoton', 'shotoff', 'foulcommit',
       'card', 'cross', 'corner', 'possession', 'B365H', 'B365D', 'B365A',
       'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA',
       'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA',
       'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA',
       'league_name_Belgium Jupiler League'], dtype=object)

In [2030]:
matches.columns.values[30:38]

array(['goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross',
       'corner', 'possession'], dtype=object)

In [2031]:
matches.columns.values[38:71]

array(['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD',
       'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD',
       'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD',
       'GBA', 'BSH', 'BSD', 'BSA', 'league_name_Belgium Jupiler League',
       'league_name_England Premier League', 'league_name_France Ligue 1'],
      dtype=object)

In [2032]:
# Create columns of all average odds for home win, draw and away win, then remove old columns
cols_odds_home = matches.columns.values[38:68:3]
matches['avg_odds_home_win'] = matches[cols_odds_home].mean(axis=1)
cols_odds_draw = matches.columns.values[39:69:3]
matches['avg_odds_draw'] = matches[cols_odds_draw].mean(axis=1)
cols_odds_away = matches.columns.values[40:70:3]
matches['avg_odds_away_win'] = matches[cols_odds_away].mean(axis=1)
matches.drop(matches.columns[38:68], axis=1, inplace=True)
matches

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,league_name_Italy Serie A,league_name_Netherlands Eredivisie,league_name_Poland Ekstraklasa,league_name_Portugal Liga ZON Sagres,league_name_Scotland Premier League,league_name_Spain LIGA BBVA,league_name_Switzerland Super League,avg_odds_home_win,avg_odds_draw,avg_odds_away_win
0,2008/2009,1,2008-08-16,492474,10000,9994,0,0,NaN,NaN,...,0,0,0,0,0,0,0,1.898889,3.250000,3.616667
1,2008/2009,1,2008-08-16,492475,9984,8635,0,3,NaN,NaN,...,0,0,0,0,0,0,0,2.462222,3.233333,2.544444
2,2008/2009,1,2008-08-16,492477,7947,9985,1,3,NaN,NaN,...,0,0,0,0,0,0,0,4.494444,3.422222,1.674444
3,2008/2009,1,2008-08-16,492479,9999,8571,2,2,NaN,NaN,...,0,0,0,0,0,0,0,1.957778,3.266667,3.400000
4,2008/2009,1,2008-08-16,492480,4049,9996,1,2,NaN,NaN,...,0,0,0,0,0,0,0,2.714444,3.272222,2.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,2015/2016,36,2016-05-25,1992225,9931,9956,0,1,42113.0,458806.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
25975,2015/2016,36,2016-05-25,1992226,7896,10190,3,0,330458.0,282287.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
25976,2015/2016,36,2016-05-25,1992227,10199,10179,2,2,42276.0,114792.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
25977,2015/2016,36,2016-05-25,1992228,10191,10192,0,3,462944.0,239959.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN


In [2033]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 52 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   season                                25979 non-null  object        
 1   stage                                 25979 non-null  int64         
 2   date                                  25979 non-null  datetime64[ns]
 3   match_api_id                          25979 non-null  int64         
 4   home_team_api_id                      25979 non-null  int64         
 5   away_team_api_id                      25979 non-null  int64         
 6   home_team_goal                        25979 non-null  int64         
 7   away_team_goal                        25979 non-null  int64         
 8   home_player_1                         24755 non-null  float64       
 9   home_player_2                         24664 non-null  float64       
 10

In [2034]:
matches.columns.values[:]

array(['season', 'stage', 'date', 'match_api_id', 'home_team_api_id',
       'away_team_api_id', 'home_team_goal', 'away_team_goal',
       'home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',
       'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',
       'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11', 'goal',
       'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner',
       'possession', 'league_name_Belgium Jupiler League',
       'league_name_England Premier League', 'league_name_France Ligue 1',
       'league_name_Germany 1. Bundesliga', 'league_name_Italy Serie A',
       'league_name_Netherlands Eredivisie',
       'league_name_Poland Ekstraklasa',
       'league_name_Portugal Liga ZON Sagres',
       'league_name_Scotland P

In [2035]:
# matches.fillna(value=np.nan, inplace=True)
# matches.fillna(matches.mean(), inplace=True)
matches

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,league_name_Italy Serie A,league_name_Netherlands Eredivisie,league_name_Poland Ekstraklasa,league_name_Portugal Liga ZON Sagres,league_name_Scotland Premier League,league_name_Spain LIGA BBVA,league_name_Switzerland Super League,avg_odds_home_win,avg_odds_draw,avg_odds_away_win
0,2008/2009,1,2008-08-16,492474,10000,9994,0,0,NaN,NaN,...,0,0,0,0,0,0,0,1.898889,3.250000,3.616667
1,2008/2009,1,2008-08-16,492475,9984,8635,0,3,NaN,NaN,...,0,0,0,0,0,0,0,2.462222,3.233333,2.544444
2,2008/2009,1,2008-08-16,492477,7947,9985,1,3,NaN,NaN,...,0,0,0,0,0,0,0,4.494444,3.422222,1.674444
3,2008/2009,1,2008-08-16,492479,9999,8571,2,2,NaN,NaN,...,0,0,0,0,0,0,0,1.957778,3.266667,3.400000
4,2008/2009,1,2008-08-16,492480,4049,9996,1,2,NaN,NaN,...,0,0,0,0,0,0,0,2.714444,3.272222,2.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,2015/2016,36,2016-05-25,1992225,9931,9956,0,1,42113.0,458806.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
25975,2015/2016,36,2016-05-25,1992226,7896,10190,3,0,330458.0,282287.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
25976,2015/2016,36,2016-05-25,1992227,10199,10179,2,2,42276.0,114792.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
25977,2015/2016,36,2016-05-25,1992228,10191,10192,0,3,462944.0,239959.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN


### Merge data from **Player** and **Player_Attributes**

In [2036]:
players = pd.read_sql("""SELECT *
                        FROM Player
                        JOIN Player_Attributes ON Player.player_api_id = Player_Attributes.player_api_id;""", conn)
players

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,id,player_fifa_api_id,player_api_id,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,1,218353,505942,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2,218353,505942,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,3,218353,505942,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,4,218353,505942,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,5,218353,505942,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,11075,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,183974,102359,39902,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,11075,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,183975,102359,39902,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,11075,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,183976,102359,39902,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,11075,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,183977,102359,39902,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [2037]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 49 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_api_id        183978 non-null  int64  
 2   player_name          183978 non-null  object 
 3   player_fifa_api_id   183978 non-null  int64  
 4   birthday             183978 non-null  object 
 5   height               183978 non-null  float64
 6   weight               183978 non-null  int64  
 7   id                   183978 non-null  int64  
 8   player_fifa_api_id   183978 non-null  int64  
 9   player_api_id        183978 non-null  int64  
 10  date                 183978 non-null  object 
 11  overall_rating       183142 non-null  float64
 12  potential            183142 non-null  float64
 13  preferred_foot       183142 non-null  object 
 14  attacking_work_rate  180748 non-null  object 
 15  defensive_work_ra

##### Clear data and prepare for ML

In [2038]:
players['birthday']= pd.to_datetime(players['birthday'])
players['date']= pd.to_datetime(players['date'])
players.drop(players.columns[[0, 3]], axis=1, inplace=True) # Drop unuuseful columns 'id' and 'player_fifa_api_id'
players = players.loc[:,~players.columns.duplicated()].copy() # Remove duplicate columns
players.sort_values(['player_name', 'date'], ascending=[True, False]) # Sort to get the latest updated player data first (multiple entries for same player_api_id)
players.drop_duplicates(subset="player_api_id", inplace=True) # Remove duplicate entries for 'player_api_id' and leave just the latest one (first after sorting)
players = pd.get_dummies(data=players, columns=['preferred_foot'])
work_rate_dictionary ={'low' : 0, 'medium' : 1, 'high' : 2}
players['defensive_work_rate'] = players['defensive_work_rate'].map(work_rate_dictionary)
players['attacking_work_rate'] = players['attacking_work_rate'].map(work_rate_dictionary)
players['defensive_work_rate']= pd.to_numeric(players['defensive_work_rate'])
players['attacking_work_rate']= pd.to_numeric(players['attacking_work_rate'])
players.drop('date', axis=1, inplace=True) # Date of player data update - not needed
# Missing player attributes may be filled by mean value
players.fillna(value=np.nan, inplace=True)
players.fillna(players.mean(), inplace=True)
players

/var/folders/cf/4_jk0m_n0bx7f_qhrc4dq8p80000gp/T/ipykernel_18446/3990885051.py:16: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  players.fillna(players.mean(), inplace=True)
/var/folders/cf/4_jk0m_n0bx7f_qhrc4dq8p80000gp/T/ipykernel_18446/3990885051.py:16: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  players.fillna(players.mean(), inplace=True)


,player_api_id,player_name,birthday,height,weight,overall_rating,potential,attacking_work_rate,defensive_work_rate,crossing,...,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot_left,preferred_foot_right
0,505942,Aaron Appindangoye,1992-02-29,182.88,187,67.0,71.0,1.000000,1.000000,49.0,...,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,0,1
5,155782,Aaron Cresswell,1989-12-15,170.18,146,74.0,76.0,2.000000,1.000000,80.0,...,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,1,0
38,162549,Aaron Doran,1991-05-13,170.18,163,65.0,67.0,1.000000,1.000000,64.0,...,23.0,22.0,24.0,16.0,11.0,12.0,9.0,13.0,0,1
64,30572,Aaron Galindo,1982-05-08,182.88,198,69.0,69.0,1.000000,1.000000,57.0,...,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0,0,1
87,23780,Aaron Hughes,1979-11-08,182.88,154,70.0,70.0,1.000000,1.000000,46.0,...,75.0,73.0,71.0,8.0,6.0,16.0,12.0,11.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183924,26357,Zoumana Camara,1979-04-03,182.88,168,73.0,73.0,0.000000,1.000000,42.0,...,76.0,77.0,72.0,14.0,10.0,11.0,8.0,12.0,0,1
183937,111182,Zsolt Laczko,1986-12-18,182.88,176,63.0,67.0,2.000000,1.000000,65.0,...,57.0,61.0,61.0,9.0,8.0,5.0,5.0,8.0,1,0
183953,36491,Zsolt Low,1979-04-29,180.34,154,68.0,73.0,1.182699,1.051888,64.0,...,67.0,71.0,59.0,15.0,13.0,5.0,6.0,6.0,1,0
183960,35506,Zurab Khizanishvili,1981-10-06,185.42,172,67.0,78.0,1.182699,1.051888,48.0,...,65.0,66.0,65.0,14.0,7.0,15.0,10.0,12.0,0,1


In [2039]:
players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11060 entries, 0 to 183968
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   player_api_id         11060 non-null  int64         
 1   player_name           11060 non-null  object        
 2   birthday              11060 non-null  datetime64[ns]
 3   height                11060 non-null  float64       
 4   weight                11060 non-null  int64         
 5   overall_rating        11060 non-null  float64       
 6   potential             11060 non-null  float64       
 7   attacking_work_rate   11060 non-null  float64       
 8   defensive_work_rate   11060 non-null  float64       
 9   crossing              11060 non-null  float64       
 10  finishing             11060 non-null  float64       
 11  heading_accuracy      11060 non-null  float64       
 12  short_passing         11060 non-null  float64       
 13  volleys        

### Merge data from **Team** and **Team_Attributes**

In [2040]:
teams = pd.read_sql("""SELECT *
                        FROM Team
                        JOIN Team_Attributes ON Team.team_api_id = Team_Attributes.team_api_id;""", conn)
teams

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,49119,9930,434,FC Aarau,AAR,1,434,9930,2010-02-22 00:00:00,60,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,49119,9930,434,FC Aarau,AAR,2,434,9930,2014-09-19 00:00:00,52,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,49119,9930,434,FC Aarau,AAR,3,434,9930,2015-09-10 00:00:00,47,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,39393,8485,77,Aberdeen,ABE,4,77,8485,2010-02-22 00:00:00,70,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,39393,8485,77,Aberdeen,ABE,5,77,8485,2011-02-22 00:00:00,47,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,3,10000,15005,SV Zulte-Waregem,ZUL,1454,15005,10000,2011-02-22 00:00:00,52,...,53,Normal,Organised,46,Medium,48,Press,53,Normal,Cover
1454,3,10000,15005,SV Zulte-Waregem,ZUL,1455,15005,10000,2012-02-22 00:00:00,54,...,50,Normal,Organised,44,Medium,55,Press,53,Normal,Cover
1455,3,10000,15005,SV Zulte-Waregem,ZUL,1456,15005,10000,2013-09-20 00:00:00,54,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover
1456,3,10000,15005,SV Zulte-Waregem,ZUL,1457,15005,10000,2014-09-19 00:00:00,54,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover


In [2041]:
teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              1458 non-null   int64  
 1   team_api_id                     1458 non-null   int64  
 2   team_fifa_api_id                1458 non-null   int64  
 3   team_long_name                  1458 non-null   object 
 4   team_short_name                 1458 non-null   object 
 5   id                              1458 non-null   int64  
 6   team_fifa_api_id                1458 non-null   int64  
 7   team_api_id                     1458 non-null   int64  
 8   date                            1458 non-null   object 
 9   buildUpPlaySpeed                1458 non-null   int64  
 10  buildUpPlaySpeedClass           1458 non-null   object 
 11  buildUpPlayDribbling            489 non-null    float64
 12  buildUpPlayDribblingClass       14

##### Clear data and prepare for ML

In [2042]:
teams['date']= pd.to_datetime(teams['date'])
teams.drop(teams.columns[[0, 2]], axis=1, inplace=True) # Drop 'id' and 'team_fifa_api_id'
# Below columns can be dropped as they are just classes for numeric values that already exist
teams.drop(['team_short_name',
            'buildUpPlaySpeedClass', 
            'buildUpPlayDribblingClass',
            'buildUpPlayPassingClass',
            'chanceCreationPassingClass',
            'chanceCreationCrossingClass',
            'chanceCreationShootingClass',
            'defencePressureClass',
            'defenceAggressionClass',
            'defenceTeamWidthClass'], axis=1, inplace=True)
teams = teams.loc[:,~teams.columns.duplicated()].copy() # Remove duplicate columns
teams.sort_values(['team_long_name', 'date'], ascending=[True, False]) # Sorting and then removing duplicates to keep just latest team data (same as with players before)
teams.drop_duplicates(subset="team_api_id", inplace=True)
teams.drop('date', axis=1, inplace=True)
teams = pd.get_dummies(data=teams, columns=['buildUpPlayPositioningClass', 'chanceCreationPositioningClass', 'defenceDefenderLineClass'])
# Missing team attributes can be filled with mean values
teams.fillna(value=np.nan, inplace=True)
teams.fillna(teams.mean(), inplace=True)
teams['buildUpPlayDribbling'] = teams['buildUpPlayDribbling'].astype('int64') # Convert from float to int
teams

/var/folders/cf/4_jk0m_n0bx7f_qhrc4dq8p80000gp/T/ipykernel_18446/89538795.py:21: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  teams.fillna(teams.mean(), inplace=True)


,team_api_id,team_long_name,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth,buildUpPlayPositioningClass_Free Form,buildUpPlayPositioningClass_Organised,chanceCreationPositioningClass_Free Form,chanceCreationPositioningClass_Organised,defenceDefenderLineClass_Cover,defenceDefenderLineClass_Offside Trap
0,9930,FC Aarau,60,47,50,60,65,55,50,55,45,0,1,0,1,1,0
3,8485,Aberdeen,70,47,70,70,70,70,60,70,70,0,1,0,1,1,0
9,8576,AC Ajaccio,60,47,40,45,35,55,30,70,30,0,1,0,1,0,1
15,8564,Milan,45,47,30,55,45,70,30,35,60,1,0,1,0,0,1
21,10215,Académica de Coimbra,30,47,30,50,60,55,30,30,30,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,10192,BSC Young Boys,55,47,55,55,70,60,55,55,60,0,1,0,1,0,1
1439,8021,Zagłębie Lubin,55,47,60,30,40,70,70,70,70,0,1,0,1,1,0
1444,8394,Real Zaragoza,30,47,30,50,35,70,40,30,45,1,0,0,1,0,1
1450,8027,Zawisza Bydgoszcz,54,51,40,67,49,52,44,47,52,0,1,0,1,1,0


In [2043]:
teams.columns.values

array(['team_api_id', 'team_long_name', 'buildUpPlaySpeed',
       'buildUpPlayDribbling', 'buildUpPlayPassing',
       'chanceCreationPassing', 'chanceCreationCrossing',
       'chanceCreationShooting', 'defencePressure', 'defenceAggression',
       'defenceTeamWidth', 'buildUpPlayPositioningClass_Free Form',
       'buildUpPlayPositioningClass_Organised',
       'chanceCreationPositioningClass_Free Form',
       'chanceCreationPositioningClass_Organised',
       'defenceDefenderLineClass_Cover',
       'defenceDefenderLineClass_Offside Trap'], dtype=object)

In [2044]:
teams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 0 to 1452
Data columns (total 17 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   team_api_id                               288 non-null    int64 
 1   team_long_name                            288 non-null    object
 2   buildUpPlaySpeed                          288 non-null    int64 
 3   buildUpPlayDribbling                      288 non-null    int64 
 4   buildUpPlayPassing                        288 non-null    int64 
 5   chanceCreationPassing                     288 non-null    int64 
 6   chanceCreationCrossing                    288 non-null    int64 
 7   chanceCreationShooting                    288 non-null    int64 
 8   defencePressure                           288 non-null    int64 
 9   defenceAggression                         288 non-null    int64 
 10  defenceTeamWidth                          288 non

In [2045]:
teams.columns.values[11] = "buildUpPlayPositioningClass_Free_Form"
teams.columns.values[13] = "chanceCreationPositioningClass_Free_Form"
teams.columns.values[16] = "defenceDefenderLineClass_Offside_Trap"

In [2046]:
teams

,team_api_id,team_long_name,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth,buildUpPlayPositioningClass_Free_Form,buildUpPlayPositioningClass_Organised,chanceCreationPositioningClass_Free_Form,chanceCreationPositioningClass_Organised,defenceDefenderLineClass_Cover,defenceDefenderLineClass_Offside_Trap
0,9930,FC Aarau,60,47,50,60,65,55,50,55,45,0,1,0,1,1,0
3,8485,Aberdeen,70,47,70,70,70,70,60,70,70,0,1,0,1,1,0
9,8576,AC Ajaccio,60,47,40,45,35,55,30,70,30,0,1,0,1,0,1
15,8564,Milan,45,47,30,55,45,70,30,35,60,1,0,1,0,0,1
21,10215,Académica de Coimbra,30,47,30,50,60,55,30,30,30,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,10192,BSC Young Boys,55,47,55,55,70,60,55,55,60,0,1,0,1,0,1
1439,8021,Zagłębie Lubin,55,47,60,30,40,70,70,70,70,0,1,0,1,1,0
1444,8394,Real Zaragoza,30,47,30,50,35,70,40,30,45,1,0,0,1,0,1
1450,8027,Zawisza Bydgoszcz,54,51,40,67,49,52,44,47,52,0,1,0,1,1,0


### Merge all data together

#### Check for NaN values in **matches**

In [2047]:
matches.isna().sum().sum()

133499

In [2048]:
matches['match_api_id'].isna().sum()

0

In [2049]:
matches.columns[8:30]

Index(['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',
       'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',
       'home_player_9', 'home_player_10', 'home_player_11', 'away_player_1',
       'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5',
       'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9',
       'away_player_10', 'away_player_11'],
      dtype='object')

Drop rows that are missing players (we cannot just put a random player)

In [2050]:
matches.dropna(subset=matches.columns[8:30], inplace=True)
matches

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,league_name_Italy Serie A,league_name_Netherlands Eredivisie,league_name_Poland Ekstraklasa,league_name_Portugal Liga ZON Sagres,league_name_Scotland Premier League,league_name_Spain LIGA BBVA,league_name_Switzerland Super League,avg_odds_home_win,avg_odds_draw,avg_odds_away_win
207,2008/2009,24,2009-02-27,493017,8203,9987,2,1,38327.0,67950.0,...,0,0,0,0,0,0,0,2.835556,3.233333,2.266667
217,2008/2009,25,2009-03-07,493027,8635,10000,2,0,34480.0,38388.0,...,0,0,0,0,0,0,0,1.354444,4.208889,7.777778
223,2008/2009,25,2009-03-08,493025,9984,8342,1,3,36835.0,37047.0,...,0,0,0,0,0,0,0,2.924444,3.172222,2.227778
225,2008/2009,26,2009-03-13,493034,8203,8635,2,1,38327.0,67950.0,...,0,0,0,0,0,0,0,4.392222,3.366667,1.727778
229,2008/2009,26,2009-03-14,493040,10000,9999,0,0,37900.0,37886.0,...,0,0,0,0,0,0,0,1.661111,3.444444,4.684444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,2015/2016,36,2016-05-25,1992225,9931,9956,0,1,42113.0,458806.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
25975,2015/2016,36,2016-05-25,1992226,7896,10190,3,0,330458.0,282287.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
25976,2015/2016,36,2016-05-25,1992227,10199,10179,2,2,42276.0,114792.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
25977,2015/2016,36,2016-05-25,1992228,10191,10192,0,3,462944.0,239959.0,...,0,0,0,0,0,0,1,NaN,NaN,NaN


In [2051]:
matches.isna().sum().sum()

69432

In [2052]:
nans_sum = 0
for column in matches.columns.values[20:]:
    col_nans = matches[column].isna().sum()
    print("Column: " + str(column) + " has NaNs: " + str(col_nans))
    nans_sum = nans_sum + col_nans

Column: away_player_2 has NaNs: 0
Column: away_player_3 has NaNs: 0
Column: away_player_4 has NaNs: 0
Column: away_player_5 has NaNs: 0
Column: away_player_6 has NaNs: 0
Column: away_player_7 has NaNs: 0
Column: away_player_8 has NaNs: 0
Column: away_player_9 has NaNs: 0
Column: away_player_10 has NaNs: 0
Column: away_player_11 has NaNs: 0
Column: goal has NaNs: 8049
Column: shoton has NaNs: 8049
Column: shotoff has NaNs: 8049
Column: foulcommit has NaNs: 8049
Column: card has NaNs: 8049
Column: cross has NaNs: 8049
Column: corner has NaNs: 8049
Column: possession has NaNs: 8049
Column: league_name_Belgium Jupiler League has NaNs: 0
Column: league_name_England Premier League has NaNs: 0
Column: league_name_France Ligue 1 has NaNs: 0
Column: league_name_Germany 1. Bundesliga has NaNs: 0
Column: league_name_Italy Serie A has NaNs: 0
Column: league_name_Netherlands Eredivisie has NaNs: 0
Column: league_name_Poland Ekstraklasa has NaNs: 0
Column: league_name_Portugal Liga ZON Sagres has Na

In [2053]:
nans_sum

69432

The only NaNs are left in the average odds columns. We need to decide if it's better to drop them or put 0 or mean. For now let's just drop them

In [2054]:
matches.dropna(subset=['avg_odds_home_win', 'avg_odds_draw', 'avg_odds_away_win'], inplace=True)
matches

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,league_name_Italy Serie A,league_name_Netherlands Eredivisie,league_name_Poland Ekstraklasa,league_name_Portugal Liga ZON Sagres,league_name_Scotland Premier League,league_name_Spain LIGA BBVA,league_name_Switzerland Super League,avg_odds_home_win,avg_odds_draw,avg_odds_away_win
207,2008/2009,24,2009-02-27,493017,8203,9987,2,1,38327.0,67950.0,...,0,0,0,0,0,0,0,2.835556,3.233333,2.266667
217,2008/2009,25,2009-03-07,493027,8635,10000,2,0,34480.0,38388.0,...,0,0,0,0,0,0,0,1.354444,4.208889,7.777778
223,2008/2009,25,2009-03-08,493025,9984,8342,1,3,36835.0,37047.0,...,0,0,0,0,0,0,0,2.924444,3.172222,2.227778
225,2008/2009,26,2009-03-13,493034,8203,8635,2,1,38327.0,67950.0,...,0,0,0,0,0,0,0,4.392222,3.366667,1.727778
229,2008/2009,26,2009-03-14,493040,10000,9999,0,0,37900.0,37886.0,...,0,0,0,0,0,0,0,1.661111,3.444444,4.684444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24551,2015/2016,38,2016-05-14,2030536,9906,9910,2,0,177126.0,38424.0,...,0,0,0,0,0,1,0,1.820000,3.731429,4.164286
24553,2015/2016,38,2016-05-15,2030531,8603,8305,2,1,196126.0,186991.0,...,0,0,0,0,0,1,0,4.854286,3.895714,1.680000
24554,2015/2016,38,2016-05-15,2030532,8558,8372,4,2,532942.0,185123.0,...,0,0,0,0,0,1,0,1.994286,3.557143,3.668571
24555,2015/2016,38,2016-05-15,2030535,9864,8306,4,1,31047.0,41470.0,...,0,0,0,0,0,1,0,1.831429,3.647143,4.247143


In [2055]:
matches.isna().sum().sum()

51784

In [2056]:
nans_sum = 0
for column in matches.columns.values[20:]:
    col_nans = matches[column].isna().sum()
    print("Column: " + str(column) + " has NaNs: " + str(col_nans))
    nans_sum = nans_sum + col_nans

Column: away_player_2 has NaNs: 0
Column: away_player_3 has NaNs: 0
Column: away_player_4 has NaNs: 0
Column: away_player_5 has NaNs: 0
Column: away_player_6 has NaNs: 0
Column: away_player_7 has NaNs: 0
Column: away_player_8 has NaNs: 0
Column: away_player_9 has NaNs: 0
Column: away_player_10 has NaNs: 0
Column: away_player_11 has NaNs: 0
Column: goal has NaNs: 6473
Column: shoton has NaNs: 6473
Column: shotoff has NaNs: 6473
Column: foulcommit has NaNs: 6473
Column: card has NaNs: 6473
Column: cross has NaNs: 6473
Column: corner has NaNs: 6473
Column: possession has NaNs: 6473
Column: league_name_Belgium Jupiler League has NaNs: 0
Column: league_name_England Premier League has NaNs: 0
Column: league_name_France Ligue 1 has NaNs: 0
Column: league_name_Germany 1. Bundesliga has NaNs: 0
Column: league_name_Italy Serie A has NaNs: 0
Column: league_name_Netherlands Eredivisie has NaNs: 0
Column: league_name_Poland Ekstraklasa has NaNs: 0
Column: league_name_Portugal Liga ZON Sagres has Na

In [2057]:
nans_sum

51784

In [2058]:
# Watch out for NaNs as player_api_id !!!
for column in matches.columns.values[8:30]:
    matches[column] = matches[column].astype('int64')
matches

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,league_name_Italy Serie A,league_name_Netherlands Eredivisie,league_name_Poland Ekstraklasa,league_name_Portugal Liga ZON Sagres,league_name_Scotland Premier League,league_name_Spain LIGA BBVA,league_name_Switzerland Super League,avg_odds_home_win,avg_odds_draw,avg_odds_away_win
207,2008/2009,24,2009-02-27,493017,8203,9987,2,1,38327,67950,...,0,0,0,0,0,0,0,2.835556,3.233333,2.266667
217,2008/2009,25,2009-03-07,493027,8635,10000,2,0,34480,38388,...,0,0,0,0,0,0,0,1.354444,4.208889,7.777778
223,2008/2009,25,2009-03-08,493025,9984,8342,1,3,36835,37047,...,0,0,0,0,0,0,0,2.924444,3.172222,2.227778
225,2008/2009,26,2009-03-13,493034,8203,8635,2,1,38327,67950,...,0,0,0,0,0,0,0,4.392222,3.366667,1.727778
229,2008/2009,26,2009-03-14,493040,10000,9999,0,0,37900,37886,...,0,0,0,0,0,0,0,1.661111,3.444444,4.684444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24551,2015/2016,38,2016-05-14,2030536,9906,9910,2,0,177126,38424,...,0,0,0,0,0,1,0,1.820000,3.731429,4.164286
24553,2015/2016,38,2016-05-15,2030531,8603,8305,2,1,196126,186991,...,0,0,0,0,0,1,0,4.854286,3.895714,1.680000
24554,2015/2016,38,2016-05-15,2030532,8558,8372,4,2,532942,185123,...,0,0,0,0,0,1,0,1.994286,3.557143,3.668571
24555,2015/2016,38,2016-05-15,2030535,9864,8306,4,1,31047,41470,...,0,0,0,0,0,1,0,1.831429,3.647143,4.247143


Now we don't have any NaNs for players or odds and still have over 19k of match data, so we can move on and merge data

#### Check for NaN values in **players**

In [2059]:
players.isna().sum().sum()

0

#### Check for NaN values in **teams**

In [2060]:
teams.isna().sum().sum()

0

#### All data has been cleaned separately, now it's time to merge it

In [2061]:
len(matches.columns)

52

In [2062]:
len(players.columns)

44

In [2063]:
players.columns.values

array(['player_api_id', 'player_name', 'birthday', 'height', 'weight',
       'overall_rating', 'potential', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes', 'preferred_foot_left', 'preferred_foot_right'],
      dtype=object)

In [2064]:
len(teams.columns)

17

In [2065]:
all_data = matches.copy(deep=True)

#### Merge **teams** data into **all_data** (all_data is a copy of **matches**)

In [2066]:
teams.head()

,team_api_id,team_long_name,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth,buildUpPlayPositioningClass_Free_Form,buildUpPlayPositioningClass_Organised,chanceCreationPositioningClass_Free_Form,chanceCreationPositioningClass_Organised,defenceDefenderLineClass_Cover,defenceDefenderLineClass_Offside_Trap
0,9930,FC Aarau,60,47,50,60,65,55,50,55,45,0,1,0,1,1,0
3,8485,Aberdeen,70,47,70,70,70,70,60,70,70,0,1,0,1,1,0
9,8576,AC Ajaccio,60,47,40,45,35,55,30,70,30,0,1,0,1,0,1
15,8564,Milan,45,47,30,55,45,70,30,35,60,1,0,1,0,0,1
21,10215,Académica de Coimbra,30,47,30,50,60,55,30,30,30,0,1,0,1,0,1


In [2067]:
all_data.head()

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,league_name_Italy Serie A,league_name_Netherlands Eredivisie,league_name_Poland Ekstraklasa,league_name_Portugal Liga ZON Sagres,league_name_Scotland Premier League,league_name_Spain LIGA BBVA,league_name_Switzerland Super League,avg_odds_home_win,avg_odds_draw,avg_odds_away_win
207,2008/2009,24,2009-02-27,493017,8203,9987,2,1,38327,67950,...,0,0,0,0,0,0,0,2.835556,3.233333,2.266667
217,2008/2009,25,2009-03-07,493027,8635,10000,2,0,34480,38388,...,0,0,0,0,0,0,0,1.354444,4.208889,7.777778
223,2008/2009,25,2009-03-08,493025,9984,8342,1,3,36835,37047,...,0,0,0,0,0,0,0,2.924444,3.172222,2.227778
225,2008/2009,26,2009-03-13,493034,8203,8635,2,1,38327,67950,...,0,0,0,0,0,0,0,4.392222,3.366667,1.727778
229,2008/2009,26,2009-03-14,493040,10000,9999,0,0,37900,37886,...,0,0,0,0,0,0,0,1.661111,3.444444,4.684444


##### Merge home team info

In [2068]:
all_data = pd.merge(left=all_data, right=teams, left_on='home_team_api_id', right_on='team_api_id', how='inner')
all_data

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth,buildUpPlayPositioningClass_Free_Form,buildUpPlayPositioningClass_Organised,chanceCreationPositioningClass_Free_Form,chanceCreationPositioningClass_Organised,defenceDefenderLineClass_Cover,defenceDefenderLineClass_Offside_Trap
0,2008/2009,24,2009-02-27,493017,8203,9987,2,1,38327,67950,...,50,60,70,60,0,1,0,1,1,0
1,2008/2009,26,2009-03-13,493034,8203,8635,2,1,38327,67950,...,50,60,70,60,0,1,0,1,1,0
2,2009/2010,3,2009-08-15,665340,8203,8342,2,1,38327,67950,...,50,60,70,60,0,1,0,1,1,0
3,2009/2010,10,2009-10-03,665417,8203,9994,2,0,39573,67950,...,50,60,70,60,0,1,0,1,1,0
4,2009/2010,12,2009-10-25,665445,8203,8635,0,2,39573,67950,...,50,60,70,60,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19645,2015/2016,29,2016-03-13,2030441,8306,8633,1,2,71498,74991,...,70,35,45,45,0,1,0,1,0,1
19646,2015/2016,31,2016-04-02,2030462,8306,10267,2,1,71498,74991,...,70,35,45,45,0,1,0,1,0,1
19647,2015/2016,33,2016-04-16,2030483,8306,9869,1,1,71498,74991,...,70,35,45,45,0,1,0,1,0,1
19648,2015/2016,35,2016-04-22,2030504,8306,8558,4,0,477498,358339,...,70,35,45,45,0,1,0,1,0,1


In [2069]:
all_data.columns.values[-18:]

array(['avg_odds_away_win', 'team_api_id', 'team_long_name',
       'buildUpPlaySpeed', 'buildUpPlayDribbling', 'buildUpPlayPassing',
       'chanceCreationPassing', 'chanceCreationCrossing',
       'chanceCreationShooting', 'defencePressure', 'defenceAggression',
       'defenceTeamWidth', 'buildUpPlayPositioningClass_Free_Form',
       'buildUpPlayPositioningClass_Organised',
       'chanceCreationPositioningClass_Free_Form',
       'chanceCreationPositioningClass_Organised',
       'defenceDefenderLineClass_Cover',
       'defenceDefenderLineClass_Offside_Trap'], dtype=object)

In [2070]:
all_data.columns.values[len(all_data.columns) - 17]

'team_api_id'

In [2071]:
index = len(all_data.columns) - 17
for column in all_data.columns.values[-17:]:
    all_data.columns.values[index] = "home_team_" + str(column)
    index += 1
all_data

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,home_team_chanceCreationShooting,home_team_defencePressure,home_team_defenceAggression,home_team_defenceTeamWidth,home_team_buildUpPlayPositioningClass_Free_Form,home_team_buildUpPlayPositioningClass_Organised,home_team_chanceCreationPositioningClass_Free_Form,home_team_chanceCreationPositioningClass_Organised,home_team_defenceDefenderLineClass_Cover,home_team_defenceDefenderLineClass_Offside_Trap
0,2008/2009,24,2009-02-27,493017,8203,9987,2,1,38327,67950,...,50,60,70,60,0,1,0,1,1,0
1,2008/2009,26,2009-03-13,493034,8203,8635,2,1,38327,67950,...,50,60,70,60,0,1,0,1,1,0
2,2009/2010,3,2009-08-15,665340,8203,8342,2,1,38327,67950,...,50,60,70,60,0,1,0,1,1,0
3,2009/2010,10,2009-10-03,665417,8203,9994,2,0,39573,67950,...,50,60,70,60,0,1,0,1,1,0
4,2009/2010,12,2009-10-25,665445,8203,8635,0,2,39573,67950,...,50,60,70,60,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19645,2015/2016,29,2016-03-13,2030441,8306,8633,1,2,71498,74991,...,70,35,45,45,0,1,0,1,0,1
19646,2015/2016,31,2016-04-02,2030462,8306,10267,2,1,71498,74991,...,70,35,45,45,0,1,0,1,0,1
19647,2015/2016,33,2016-04-16,2030483,8306,9869,1,1,71498,74991,...,70,35,45,45,0,1,0,1,0,1
19648,2015/2016,35,2016-04-22,2030504,8306,8558,4,0,477498,358339,...,70,35,45,45,0,1,0,1,0,1


In [2072]:
all_data.columns.values[-20:]

array(['avg_odds_home_win', 'avg_odds_draw', 'avg_odds_away_win',
       'home_team_team_api_id', 'home_team_team_long_name',
       'home_team_buildUpPlaySpeed', 'home_team_buildUpPlayDribbling',
       'home_team_buildUpPlayPassing', 'home_team_chanceCreationPassing',
       'home_team_chanceCreationCrossing',
       'home_team_chanceCreationShooting', 'home_team_defencePressure',
       'home_team_defenceAggression', 'home_team_defenceTeamWidth',
       'home_team_buildUpPlayPositioningClass_Free_Form',
       'home_team_buildUpPlayPositioningClass_Organised',
       'home_team_chanceCreationPositioningClass_Free_Form',
       'home_team_chanceCreationPositioningClass_Organised',
       'home_team_defenceDefenderLineClass_Cover',
       'home_team_defenceDefenderLineClass_Offside_Trap'], dtype=object)

##### Merge away team info

In [2073]:
all_data = pd.merge(left=all_data, right=teams, left_on='away_team_api_id', right_on='team_api_id', how='inner')
all_data

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth,buildUpPlayPositioningClass_Free_Form,buildUpPlayPositioningClass_Organised,chanceCreationPositioningClass_Free_Form,chanceCreationPositioningClass_Organised,defenceDefenderLineClass_Cover,defenceDefenderLineClass_Offside_Trap
0,2008/2009,24,2009-02-27,493017,8203,9987,2,1,38327,67950,...,60,70,65,70,0,1,0,1,1,0
1,2010/2011,10,2010-10-01,838587,8203,9987,2,2,39573,67950,...,60,70,65,70,0,1,0,1,1,0
2,2011/2012,22,2012-01-25,1032865,8203,9987,3,2,39573,80184,...,60,70,65,70,0,1,0,1,1,0
3,2012/2013,24,2013-01-26,1224173,8203,9987,2,1,13131,67959,...,60,70,65,70,0,1,0,1,1,0
4,2014/2015,1,2014-07-27,1717822,8203,9987,3,1,13131,67896,...,60,70,65,70,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008/2009,17,2009-01-04,530366,8479,8388,3,1,33958,37434,...,55,30,55,55,0,1,0,1,0,1
19608,2008/2009,15,2008-12-14,530352,9864,8388,2,0,33827,33843,...,55,30,55,55,0,1,0,1,0,1
19609,2008/2009,13,2008-11-30,530332,8315,8388,2,0,33764,33025,...,55,30,55,55,0,1,0,1,0,1
19610,2008/2009,19,2009-01-17,530449,8302,8388,1,0,71498,70255,...,55,30,55,55,0,1,0,1,0,1


In [2074]:
all_data.columns.values[-20:]

array(['home_team_chanceCreationPositioningClass_Organised_x',
       'home_team_defenceDefenderLineClass_Cover_x',
       'home_team_defenceDefenderLineClass_Offside_Trap_x', 'team_api_id',
       'team_long_name', 'buildUpPlaySpeed', 'buildUpPlayDribbling',
       'buildUpPlayPassing', 'chanceCreationPassing',
       'chanceCreationCrossing', 'chanceCreationShooting',
       'defencePressure', 'defenceAggression', 'defenceTeamWidth',
       'buildUpPlayPositioningClass_Free_Form',
       'buildUpPlayPositioningClass_Organised',
       'chanceCreationPositioningClass_Free_Form',
       'chanceCreationPositioningClass_Organised',
       'defenceDefenderLineClass_Cover',
       'defenceDefenderLineClass_Offside_Trap'], dtype=object)

In [2075]:
index = len(all_data.columns) - 17
for column in all_data.columns.values[-17:]:
    all_data.columns.values[index] = "away_team_" + str(column)
    index += 1
all_data

,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,...,away_team_chanceCreationShooting,away_team_defencePressure,away_team_defenceAggression,away_team_defenceTeamWidth,away_team_buildUpPlayPositioningClass_Free_Form,away_team_buildUpPlayPositioningClass_Organised,away_team_chanceCreationPositioningClass_Free_Form,away_team_chanceCreationPositioningClass_Organised,away_team_defenceDefenderLineClass_Cover,away_team_defenceDefenderLineClass_Offside_Trap
0,2008/2009,24,2009-02-27,493017,8203,9987,2,1,38327,67950,...,60,70,65,70,0,1,0,1,1,0
1,2010/2011,10,2010-10-01,838587,8203,9987,2,2,39573,67950,...,60,70,65,70,0,1,0,1,1,0
2,2011/2012,22,2012-01-25,1032865,8203,9987,3,2,39573,80184,...,60,70,65,70,0,1,0,1,1,0
3,2012/2013,24,2013-01-26,1224173,8203,9987,2,1,13131,67959,...,60,70,65,70,0,1,0,1,1,0
4,2014/2015,1,2014-07-27,1717822,8203,9987,3,1,13131,67896,...,60,70,65,70,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008/2009,17,2009-01-04,530366,8479,8388,3,1,33958,37434,...,55,30,55,55,0,1,0,1,0,1
19608,2008/2009,15,2008-12-14,530352,9864,8388,2,0,33827,33843,...,55,30,55,55,0,1,0,1,0,1
19609,2008/2009,13,2008-11-30,530332,8315,8388,2,0,33764,33025,...,55,30,55,55,0,1,0,1,0,1
19610,2008/2009,19,2009-01-17,530449,8302,8388,1,0,71498,70255,...,55,30,55,55,0,1,0,1,0,1


In [2076]:
all_data.columns.values[-20:]

array(['home_team_chanceCreationPositioningClass_Organised_x',
       'home_team_defenceDefenderLineClass_Cover_x',
       'home_team_defenceDefenderLineClass_Offside_Trap_x',
       'away_team_team_api_id', 'away_team_team_long_name',
       'away_team_buildUpPlaySpeed', 'away_team_buildUpPlayDribbling',
       'away_team_buildUpPlayPassing', 'away_team_chanceCreationPassing',
       'away_team_chanceCreationCrossing',
       'away_team_chanceCreationShooting', 'away_team_defencePressure',
       'away_team_defenceAggression', 'away_team_defenceTeamWidth',
       'away_team_buildUpPlayPositioningClass_Free_Form',
       'away_team_buildUpPlayPositioningClass_Organised',
       'away_team_chanceCreationPositioningClass_Free_Form',
       'away_team_chanceCreationPositioningClass_Organised',
       'away_team_defenceDefenderLineClass_Cover',
       'away_team_defenceDefenderLineClass_Offside_Trap'], dtype=object)

In [2077]:
all_data.columns.values[-36:]

array(['avg_odds_draw', 'avg_odds_away_win', 'home_team_team_api_id_x',
       'home_team_team_long_name_x', 'home_team_buildUpPlaySpeed_x',
       'home_team_buildUpPlayDribbling_x',
       'home_team_buildUpPlayPassing_x',
       'home_team_chanceCreationPassing_x',
       'home_team_chanceCreationCrossing_x',
       'home_team_chanceCreationShooting_x',
       'home_team_defencePressure_x', 'home_team_defenceAggression_x',
       'home_team_defenceTeamWidth_x',
       'home_team_buildUpPlayPositioningClass_Free_Form_x',
       'home_team_buildUpPlayPositioningClass_Organised_x',
       'home_team_chanceCreationPositioningClass_Free_Form_x',
       'home_team_chanceCreationPositioningClass_Organised_x',
       'home_team_defenceDefenderLineClass_Cover_x',
       'home_team_defenceDefenderLineClass_Offside_Trap_x',
       'away_team_team_api_id', 'away_team_team_long_name',
       'away_team_buildUpPlaySpeed', 'away_team_buildUpPlayDribbling',
       'away_team_buildUpPlayPassing', 'a

##### Drop old home_api_id and _away_api_id

In [2078]:
all_data.drop(['home_team_api_id', 'away_team_api_id'], axis=1, inplace=True)
all_data

,season,stage,date,match_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,...,away_team_chanceCreationShooting,away_team_defencePressure,away_team_defenceAggression,away_team_defenceTeamWidth,away_team_buildUpPlayPositioningClass_Free_Form,away_team_buildUpPlayPositioningClass_Organised,away_team_chanceCreationPositioningClass_Free_Form,away_team_chanceCreationPositioningClass_Organised,away_team_defenceDefenderLineClass_Cover,away_team_defenceDefenderLineClass_Offside_Trap
0,2008/2009,24,2009-02-27,493017,2,1,38327,67950,67958,67959,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010/2011,10,2010-10-01,838587,2,2,39573,67950,39389,80184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011/2012,22,2012-01-25,1032865,3,2,39573,80184,67941,38956,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012/2013,24,2013-01-26,1224173,2,1,13131,67959,39389,396929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014/2015,1,2014-07-27,1717822,3,1,13131,67896,396929,140932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008/2009,17,2009-01-04,530366,3,1,33958,37434,37440,37423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19608,2008/2009,15,2008-12-14,530352,2,0,33827,33843,25913,41707,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19609,2008/2009,13,2008-11-30,530332,2,0,33764,33025,33993,159881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19610,2008/2009,19,2009-01-17,530449,1,0,71498,70255,38138,33990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Merge **players** data into **all_data**

In [2079]:
# TEST FOR 1 PLAYER DATA
all_data = pd.merge(left=all_data, right=players, left_on='home_player_1', right_on='player_api_id', how='inner')
all_data

,season,stage,date,match_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,...,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot_left,preferred_foot_right
0,2008/2009,24,2009-02-27,493017,2,1,38327,67950,67958,67959,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
1,2008/2009,26,2009-03-13,493034,2,1,38327,67950,67958,38801,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
2,2014/2015,11,2014-10-18,1717903,1,1,38327,38956,67959,300977,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
3,2009/2010,3,2009-08-15,665340,2,1,38327,67950,6800,39389,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
4,2014/2015,5,2014-08-24,1717854,3,1,38327,140932,241501,67896,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2015/2016,35,2016-04-22,2030504,4,0,477498,358339,643570,291635,...,12.0,14.0,13.0,66.0,71.0,56.0,69.0,74.0,1,0
19608,2014/2015,26,2015-03-09,1778284,1,2,38559,425595,213812,38148,...,25.0,25.0,25.0,64.0,72.0,65.0,73.0,62.0,0,1
19609,2014/2015,37,2015-05-17,1778399,1,2,38559,178574,309537,213714,...,25.0,25.0,25.0,64.0,72.0,65.0,73.0,62.0,0,1
19610,2014/2015,24,2015-02-21,1778265,1,2,38559,425595,280638,38148,...,25.0,25.0,25.0,64.0,72.0,65.0,73.0,62.0,0,1


In [2080]:
all_data.columns.values[-45:]

array(['away_team_defenceDefenderLineClass_Offside_Trap', 'player_api_id',
       'player_name', 'birthday', 'height', 'weight', 'overall_rating',
       'potential', 'attacking_work_rate', 'defensive_work_rate',
       'crossing', 'finishing', 'heading_accuracy', 'short_passing',
       'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping',
       'stamina', 'strength', 'long_shots', 'aggression', 'interceptions',
       'positioning', 'vision', 'penalties', 'marking', 'standing_tackle',
       'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking',
       'gk_positioning', 'gk_reflexes', 'preferred_foot_left',
       'preferred_foot_right'], dtype=object)

In [2081]:
index = len(all_data.columns) - 44
for column in all_data.columns.values[-44:]:
    all_data.columns.values[index] = "home_player_1_" + str(column)
    index += 1
all_data

,season,stage,date,match_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,...,home_player_1_marking,home_player_1_standing_tackle,home_player_1_sliding_tackle,home_player_1_gk_diving,home_player_1_gk_handling,home_player_1_gk_kicking,home_player_1_gk_positioning,home_player_1_gk_reflexes,home_player_1_preferred_foot_left,home_player_1_preferred_foot_right
0,2008/2009,24,2009-02-27,493017,2,1,38327,67950,67958,67959,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
1,2008/2009,26,2009-03-13,493034,2,1,38327,67950,67958,38801,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
2,2014/2015,11,2014-10-18,1717903,1,1,38327,38956,67959,300977,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
3,2009/2010,3,2009-08-15,665340,2,1,38327,67950,6800,39389,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
4,2014/2015,5,2014-08-24,1717854,3,1,38327,140932,241501,67896,...,13.0,13.0,13.0,69.0,66.0,65.0,66.0,73.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2015/2016,35,2016-04-22,2030504,4,0,477498,358339,643570,291635,...,12.0,14.0,13.0,66.0,71.0,56.0,69.0,74.0,1,0
19608,2014/2015,26,2015-03-09,1778284,1,2,38559,425595,213812,38148,...,25.0,25.0,25.0,64.0,72.0,65.0,73.0,62.0,0,1
19609,2014/2015,37,2015-05-17,1778399,1,2,38559,178574,309537,213714,...,25.0,25.0,25.0,64.0,72.0,65.0,73.0,62.0,0,1
19610,2014/2015,24,2015-02-21,1778265,1,2,38559,425595,280638,38148,...,25.0,25.0,25.0,64.0,72.0,65.0,73.0,62.0,0,1


In [2082]:
all_data.columns.values[-45:]

array(['away_team_defenceDefenderLineClass_Offside_Trap',
       'home_player_1_player_api_id', 'home_player_1_player_name',
       'home_player_1_birthday', 'home_player_1_height',
       'home_player_1_weight', 'home_player_1_overall_rating',
       'home_player_1_potential', 'home_player_1_attacking_work_rate',
       'home_player_1_defensive_work_rate', 'home_player_1_crossing',
       'home_player_1_finishing', 'home_player_1_heading_accuracy',
       'home_player_1_short_passing', 'home_player_1_volleys',
       'home_player_1_dribbling', 'home_player_1_curve',
       'home_player_1_free_kick_accuracy', 'home_player_1_long_passing',
       'home_player_1_ball_control', 'home_player_1_acceleration',
       'home_player_1_sprint_speed', 'home_player_1_agility',
       'home_player_1_reactions', 'home_player_1_balance',
       'home_player_1_shot_power', 'home_player_1_jumping',
       'home_player_1_stamina', 'home_player_1_strength',
       'home_player_1_long_shots', 'home_player

In [2083]:
all_data.columns.values[7:28]

array(['home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
       'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1',
       'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5',
       'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9',
       'away_player_10', 'away_player_11'], dtype=object)

In [2084]:
# Merge rest of players data
for player in all_data.columns.values[7:28]:
    all_data = pd.merge(left=all_data, right=players, left_on=player, right_on='player_api_id', how='inner')
    index = len(all_data.columns) - 44
    for column in all_data.columns.values[-44:]:
        all_data.columns.values[index] = str(player) + "_" + str(column)
        index += 1

In [2085]:
all_data

,season,stage,date,match_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,...,away_player_11_marking,away_player_11_standing_tackle,away_player_11_sliding_tackle,away_player_11_gk_diving,away_player_11_gk_handling,away_player_11_gk_kicking,away_player_11_gk_positioning,away_player_11_gk_reflexes,away_player_11_preferred_foot_left,away_player_11_preferred_foot_right
0,2008/2009,24,2009-02-27,493017,2,1,38327,67950,67958,67959,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
1,2010/2011,22,2010-12-30,838754,2,2,30934,25465,95609,94030,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
2,2010/2011,8,2010-09-21,838558,0,2,38962,38347,3329,149150,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
3,2010/2011,6,2010-09-12,838528,2,2,37990,36832,27508,42594,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
4,2010/2011,30,2011-03-20,838876,0,1,36835,38342,27110,38789,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008/2009,28,2009-04-17,499562,4,0,27299,27303,27461,30704,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0
19608,2008/2009,12,2008-11-08,499419,3,0,30820,31290,34420,30902,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0
19609,2008/2009,20,2009-02-15,499489,1,1,27358,30255,71399,20396,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0
19610,2008/2009,18,2009-01-31,499478,2,0,30703,39841,59582,27411,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0


In [2086]:
all_data.columns.values[6:28]

array(['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',
       'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',
       'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11'], dtype=object)

In [2087]:
# Delete old home_player... and away_player...
all_data.drop(all_data.columns.values[6:28], axis=1, inplace=True)
all_data

,season,stage,date,match_api_id,home_team_goal,away_team_goal,goal,shoton,shotoff,foulcommit,...,away_player_11_marking,away_player_11_standing_tackle,away_player_11_sliding_tackle,away_player_11_gk_diving,away_player_11_gk_handling,away_player_11_gk_kicking,away_player_11_gk_positioning,away_player_11_gk_reflexes,away_player_11_preferred_foot_left,away_player_11_preferred_foot_right
0,2008/2009,24,2009-02-27,493017,2,1,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
1,2010/2011,22,2010-12-30,838754,2,2,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
2,2010/2011,8,2010-09-21,838558,0,2,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
3,2010/2011,6,2010-09-12,838528,2,2,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
4,2010/2011,30,2011-03-20,838876,0,1,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008/2009,28,2009-04-17,499562,4,0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0
19608,2008/2009,12,2008-11-08,499419,3,0,<goal><value><comment>p</comment><stats><penal...,<shoton />,<shotoff />,<foulcommit />,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0
19609,2008/2009,20,2009-02-15,499489,1,1,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0
19610,2008/2009,18,2009-01-31,499478,2,0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0


In [2088]:
all_data.columns.values[:40]

array(['season', 'stage', 'date', 'match_api_id', 'home_team_goal',
       'away_team_goal', 'goal', 'shoton', 'shotoff', 'foulcommit',
       'card', 'cross', 'corner', 'possession',
       'league_name_Belgium Jupiler League',
       'league_name_England Premier League', 'league_name_France Ligue 1',
       'league_name_Germany 1. Bundesliga', 'league_name_Italy Serie A',
       'league_name_Netherlands Eredivisie',
       'league_name_Poland Ekstraklasa',
       'league_name_Portugal Liga ZON Sagres',
       'league_name_Scotland Premier League',
       'league_name_Spain LIGA BBVA',
       'league_name_Switzerland Super League', 'avg_odds_home_win',
       'avg_odds_draw', 'avg_odds_away_win', 'home_team_team_api_id_x',
       'home_team_team_long_name_x', 'home_team_buildUpPlaySpeed_x',
       'home_team_buildUpPlayDribbling_x',
       'home_team_buildUpPlayPassing_x',
       'home_team_chanceCreationPassing_x',
       'home_team_chanceCreationCrossing_x',
       'home_team_chance

In [2089]:
all_data.columns.values[40:80]

array(['home_team_buildUpPlayPositioningClass_Organised_x',
       'home_team_chanceCreationPositioningClass_Free_Form_x',
       'home_team_chanceCreationPositioningClass_Organised_x',
       'home_team_defenceDefenderLineClass_Cover_x',
       'home_team_defenceDefenderLineClass_Offside_Trap_x',
       'away_team_team_api_id', 'away_team_team_long_name',
       'away_team_buildUpPlaySpeed', 'away_team_buildUpPlayDribbling',
       'away_team_buildUpPlayPassing', 'away_team_chanceCreationPassing',
       'away_team_chanceCreationCrossing',
       'away_team_chanceCreationShooting', 'away_team_defencePressure',
       'away_team_defenceAggression', 'away_team_defenceTeamWidth',
       'away_team_buildUpPlayPositioningClass_Free_Form',
       'away_team_buildUpPlayPositioningClass_Organised',
       'away_team_chanceCreationPositioningClass_Free_Form',
       'away_team_chanceCreationPositioningClass_Organised',
       'away_team_defenceDefenderLineClass_Cover',
       'away_team_defenc

In [2090]:
all_data.columns.values[80:120]

array(['home_player_1_ball_control_x', 'home_player_1_acceleration_x',
       'home_player_1_sprint_speed_x', 'home_player_1_agility_x',
       'home_player_1_reactions_x', 'home_player_1_balance_x',
       'home_player_1_shot_power_x', 'home_player_1_jumping_x',
       'home_player_1_stamina_x', 'home_player_1_strength_x',
       'home_player_1_long_shots_x', 'home_player_1_aggression_x',
       'home_player_1_interceptions_x', 'home_player_1_positioning_x',
       'home_player_1_vision_x', 'home_player_1_penalties_x',
       'home_player_1_marking_x', 'home_player_1_standing_tackle_x',
       'home_player_1_sliding_tackle_x', 'home_player_1_gk_diving_x',
       'home_player_1_gk_handling_x', 'home_player_1_gk_kicking_x',
       'home_player_1_gk_positioning_x', 'home_player_1_gk_reflexes_x',
       'home_player_1_preferred_foot_left_x',
       'home_player_1_preferred_foot_right_x',
       'home_player_2_player_api_id', 'home_player_2_player_name',
       'home_player_2_birthday', 'h

In [2091]:
all_data.columns.values[-40:]

array(['away_player_11_weight', 'away_player_11_overall_rating',
       'away_player_11_potential', 'away_player_11_attacking_work_rate',
       'away_player_11_defensive_work_rate', 'away_player_11_crossing',
       'away_player_11_finishing', 'away_player_11_heading_accuracy',
       'away_player_11_short_passing', 'away_player_11_volleys',
       'away_player_11_dribbling', 'away_player_11_curve',
       'away_player_11_free_kick_accuracy', 'away_player_11_long_passing',
       'away_player_11_ball_control', 'away_player_11_acceleration',
       'away_player_11_sprint_speed', 'away_player_11_agility',
       'away_player_11_reactions', 'away_player_11_balance',
       'away_player_11_shot_power', 'away_player_11_jumping',
       'away_player_11_stamina', 'away_player_11_strength',
       'away_player_11_long_shots', 'away_player_11_aggression',
       'away_player_11_interceptions', 'away_player_11_positioning',
       'away_player_11_vision', 'away_player_11_penalties',
       'awa

In [2092]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19612 entries, 0 to 19611
Columns: 1030 entries, season to away_player_11_preferred_foot_right
dtypes: datetime64[ns](23), float64(856), int64(58), object(32), uint8(61)
memory usage: 146.3+ MB


In [2093]:
all_data.head()

,season,stage,date,match_api_id,home_team_goal,away_team_goal,goal,shoton,shotoff,foulcommit,...,away_player_11_marking,away_player_11_standing_tackle,away_player_11_sliding_tackle,away_player_11_gk_diving,away_player_11_gk_handling,away_player_11_gk_kicking,away_player_11_gk_positioning,away_player_11_gk_reflexes,away_player_11_preferred_foot_left,away_player_11_preferred_foot_right
0,2008/2009,24,2009-02-27,493017,2,1,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
1,2010/2011,22,2010-12-30,838754,2,2,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
2,2010/2011,8,2010-09-21,838558,0,2,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
3,2010/2011,6,2010-09-12,838528,2,2,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
4,2010/2011,30,2011-03-20,838876,0,1,None,None,None,None,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1


##### Now let's save this DataFrame for analytical usage

In [2094]:
all_data.to_pickle("data/analytical_data.pkl")

# Further preparaption for ML training

In [2095]:
ml_data = all_data.copy(deep=True)

In [2096]:
ml_data.columns.values[:40]

array(['season', 'stage', 'date', 'match_api_id', 'home_team_goal',
       'away_team_goal', 'goal', 'shoton', 'shotoff', 'foulcommit',
       'card', 'cross', 'corner', 'possession',
       'league_name_Belgium Jupiler League',
       'league_name_England Premier League', 'league_name_France Ligue 1',
       'league_name_Germany 1. Bundesliga', 'league_name_Italy Serie A',
       'league_name_Netherlands Eredivisie',
       'league_name_Poland Ekstraklasa',
       'league_name_Portugal Liga ZON Sagres',
       'league_name_Scotland Premier League',
       'league_name_Spain LIGA BBVA',
       'league_name_Switzerland Super League', 'avg_odds_home_win',
       'avg_odds_draw', 'avg_odds_away_win', 'home_team_team_api_id_x',
       'home_team_team_long_name_x', 'home_team_buildUpPlaySpeed_x',
       'home_team_buildUpPlayDribbling_x',
       'home_team_buildUpPlayPassing_x',
       'home_team_chanceCreationPassing_x',
       'home_team_chanceCreationCrossing_x',
       'home_team_chance

In [2097]:
ml_data.columns.values[6:14]

array(['goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross',
       'corner', 'possession'], dtype=object)

We will drop the match events that we do not want to predict, leave only home_team_goal and away_team_goal

In [2098]:
ml_data.drop(ml_data.columns.values[6:14], axis=1, inplace=True)

In [2099]:
ml_data

,season,stage,date,match_api_id,home_team_goal,away_team_goal,league_name_Belgium Jupiler League,league_name_England Premier League,league_name_France Ligue 1,league_name_Germany 1. Bundesliga,...,away_player_11_marking,away_player_11_standing_tackle,away_player_11_sliding_tackle,away_player_11_gk_diving,away_player_11_gk_handling,away_player_11_gk_kicking,away_player_11_gk_positioning,away_player_11_gk_reflexes,away_player_11_preferred_foot_left,away_player_11_preferred_foot_right
0,2008/2009,24,2009-02-27,493017,2,1,1,0,0,0,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
1,2010/2011,22,2010-12-30,838754,2,2,1,0,0,0,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
2,2010/2011,8,2010-09-21,838558,0,2,1,0,0,0,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
3,2010/2011,6,2010-09-12,838528,2,2,1,0,0,0,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
4,2010/2011,30,2011-03-20,838876,0,1,1,0,0,0,...,33.0,39.0,40.0,11.0,8.0,13.0,9.0,13.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008/2009,28,2009-04-17,499562,4,0,0,0,0,1,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0
19608,2008/2009,12,2008-11-08,499419,3,0,0,0,0,1,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0
19609,2008/2009,20,2009-02-15,499489,1,1,0,0,0,1,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0
19610,2008/2009,18,2009-01-31,499478,2,0,0,0,0,1,...,22.0,29.0,21.0,11.0,10.0,13.0,9.0,9.0,1,0


In [2100]:
# 'birthday' columns for all players
birthday_cols = ml_data.columns.values[56::44]
birthday_cols

array(['home_player_1_birthday_x', 'home_player_2_birthday',
       'home_player_3_birthday', 'home_player_4_birthday',
       'home_player_5_birthday', 'home_player_6_birthday',
       'home_player_7_birthday', 'home_player_8_birthday',
       'home_player_9_birthday', 'home_player_10_birthday',
       'home_player_11_birthday', 'away_player_1_birthday',
       'away_player_2_birthday', 'away_player_3_birthday',
       'away_player_4_birthday', 'away_player_5_birthday',
       'away_player_6_birthday', 'away_player_7_birthday',
       'away_player_8_birthday', 'away_player_9_birthday',
       'away_player_10_birthday', 'away_player_11_birthday'], dtype=object)

In [2101]:
len(birthday_cols)

22

We will change 'birthday' to 'age' at the time of the match for each player as the difference between match 'date' and 'birthday' (better to train Neural Network)

In [2102]:
index = 56
for birthday_col in birthday_cols:
    # Calculate new value
    new_col_name = birthday_col.replace('birthday', 'age')
    ml_data[new_col_name] = (ml_data['date'] - ml_data[birthday_col]).astype('timedelta64[Y]').astype('int')
    index += 44
ml_data.drop(birthday_cols, axis=1, inplace=True)

In [2103]:
ml_data

,season,stage,date,match_api_id,home_team_goal,away_team_goal,league_name_Belgium Jupiler League,league_name_England Premier League,league_name_France Ligue 1,league_name_Germany 1. Bundesliga,...,away_player_2_age,away_player_3_age,away_player_4_age,away_player_5_age,away_player_6_age,away_player_7_age,away_player_8_age,away_player_9_age,away_player_10_age,away_player_11_age
0,2008/2009,24,2009-02-27,493017,2,1,1,0,0,0,...,27,18,21,21,22,23,27,27,26,19
1,2010/2011,22,2010-12-30,838754,2,2,1,0,0,0,...,27,28,25,23,25,22,19,29,23,21
2,2010/2011,8,2010-09-21,838558,0,2,1,0,0,0,...,27,28,24,22,25,22,20,19,22,21
3,2010/2011,6,2010-09-12,838528,2,2,1,0,0,0,...,27,28,24,22,29,25,22,19,22,21
4,2010/2011,30,2011-03-20,838876,0,1,1,0,0,0,...,27,25,23,23,29,23,25,19,23,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008/2009,28,2009-04-17,499562,4,0,0,0,0,1,...,28,29,33,26,32,26,32,29,26,29
19608,2008/2009,12,2008-11-08,499419,3,0,0,0,0,1,...,27,32,28,25,30,30,27,31,25,28
19609,2008/2009,20,2009-02-15,499489,1,1,0,0,0,1,...,28,28,33,26,30,30,29,32,25,29
19610,2008/2009,18,2009-01-31,499478,2,0,0,0,0,1,...,28,28,33,26,30,30,27,32,27,29


In [2104]:
ml_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19612 entries, 0 to 19611
Columns: 1022 entries, season to away_player_11_age
dtypes: datetime64[ns](1), float64(856), int64(80), object(24), uint8(61)
memory usage: 145.1+ MB


In [2105]:
ml_data.dtypes

season                        object
stage                          int64
date                  datetime64[ns]
match_api_id                   int64
home_team_goal                 int64
                           ...      
away_player_7_age              int64
away_player_8_age              int64
away_player_9_age              int64
away_player_10_age             int64
away_player_11_age             int64
Length: 1022, dtype: object

In [2106]:
ml_data.dtypes[ml_data.dtypes != 'int64'][ml_data.dtypes != 'float64'][ml_data.dtypes != 'uint8']

season                                 object
date                           datetime64[ns]
home_team_team_long_name_x             object
home_player_1_player_name_x            object
home_player_2_player_name              object
home_player_3_player_name              object
home_player_4_player_name              object
home_player_5_player_name              object
home_player_6_player_name              object
home_player_7_player_name              object
home_player_8_player_name              object
home_player_9_player_name              object
home_player_10_player_name             object
home_player_11_player_name             object
away_player_1_player_name              object
away_player_2_player_name              object
away_player_3_player_name              object
away_player_4_player_name              object
away_player_5_player_name              object
away_player_6_player_name              object
away_player_7_player_name              object
away_player_8_player_name         

In [2107]:
ml_data.columns.values[:40]

/Users/almazur/miniconda3/envs/nn/lib/python3.10/site-packages/IPython/core/displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


array(['season', 'stage', 'date', 'match_api_id', 'home_team_goal',
       'away_team_goal', 'league_name_Belgium Jupiler League',
       'league_name_England Premier League', 'league_name_France Ligue 1',
       'league_name_Germany 1. Bundesliga', 'league_name_Italy Serie A',
       'league_name_Netherlands Eredivisie',
       'league_name_Poland Ekstraklasa',
       'league_name_Portugal Liga ZON Sagres',
       'league_name_Scotland Premier League',
       'league_name_Spain LIGA BBVA',
       'league_name_Switzerland Super League', 'avg_odds_home_win',
       'avg_odds_draw', 'avg_odds_away_win', 'home_team_team_api_id_x',
       'home_team_team_long_name_x', 'home_team_buildUpPlaySpeed_x',
       'home_team_buildUpPlayDribbling_x',
       'home_team_buildUpPlayPassing_x',
       'home_team_chanceCreationPassing_x',
       'home_team_chanceCreationCrossing_x',
       'home_team_chanceCreationShooting_x',
       'home_team_defencePressure_x', 'home_team_defenceAggression_x',
      

Let's delete data that is not usefull for model training, like names, as we will keep ids

In [2108]:
names_cols = ml_data.dtypes[ml_data.dtypes != 'int64'][ml_data.dtypes != 'float64'][ml_data.dtypes != 'uint8'][2:]
names_cols

home_team_team_long_name_x     object
home_player_1_player_name_x    object
home_player_2_player_name      object
home_player_3_player_name      object
home_player_4_player_name      object
home_player_5_player_name      object
home_player_6_player_name      object
home_player_7_player_name      object
home_player_8_player_name      object
home_player_9_player_name      object
home_player_10_player_name     object
home_player_11_player_name     object
away_player_1_player_name      object
away_player_2_player_name      object
away_player_3_player_name      object
away_player_4_player_name      object
away_player_5_player_name      object
away_player_6_player_name      object
away_player_7_player_name      object
away_player_8_player_name      object
away_player_9_player_name      object
away_player_10_player_name     object
away_player_11_player_name     object
dtype: object

In [2109]:
ml_data.drop(['home_team_team_long_name_x',
              # 'away_team_long_name',
              'home_player_1_player_name_x',
              'home_player_2_player_name',
              'home_player_3_player_name',
              'home_player_4_player_name',
              'home_player_5_player_name',
              'home_player_6_player_name',
              'home_player_7_player_name',
              'home_player_8_player_name',
              'home_player_9_player_name',
              'home_player_10_player_name',
              'home_player_11_player_name',
              'away_player_1_player_name',
              'away_player_2_player_name',
              'away_player_3_player_name',
              'away_player_4_player_name',
              'away_player_5_player_name',
              'away_player_6_player_name',
              'away_player_7_player_name',
              'away_player_8_player_name',
              'away_player_9_player_name',
              'away_player_10_player_name',
              'away_player_11_player_name'], axis=1, inplace=True)

In [2110]:
ml_data.dtypes[ml_data.dtypes != 'int64'][ml_data.dtypes != 'float64'][ml_data.dtypes != 'uint8']

season            object
date      datetime64[ns]
dtype: object

In [2111]:
ml_data.dtypes

season                        object
stage                          int64
date                  datetime64[ns]
match_api_id                   int64
home_team_goal                 int64
                           ...      
away_player_7_age              int64
away_player_8_age              int64
away_player_9_age              int64
away_player_10_age             int64
away_player_11_age             int64
Length: 999, dtype: object

In [2112]:
ml_data.columns.values[:40]

array(['season', 'stage', 'date', 'match_api_id', 'home_team_goal',
       'away_team_goal', 'league_name_Belgium Jupiler League',
       'league_name_England Premier League', 'league_name_France Ligue 1',
       'league_name_Germany 1. Bundesliga', 'league_name_Italy Serie A',
       'league_name_Netherlands Eredivisie',
       'league_name_Poland Ekstraklasa',
       'league_name_Portugal Liga ZON Sagres',
       'league_name_Scotland Premier League',
       'league_name_Spain LIGA BBVA',
       'league_name_Switzerland Super League', 'avg_odds_home_win',
       'avg_odds_draw', 'avg_odds_away_win', 'home_team_team_api_id_x',
       'home_team_buildUpPlaySpeed_x', 'home_team_buildUpPlayDribbling_x',
       'home_team_buildUpPlayPassing_x',
       'home_team_chanceCreationPassing_x',
       'home_team_chanceCreationCrossing_x',
       'home_team_chanceCreationShooting_x',
       'home_team_defencePressure_x', 'home_team_defenceAggression_x',
       'home_team_defenceTeamWidth_x',
    

In [2113]:
ml_data.columns.values[40:80]

array(['away_team_buildUpPlayPassing', 'away_team_chanceCreationPassing',
       'away_team_chanceCreationCrossing',
       'away_team_chanceCreationShooting', 'away_team_defencePressure',
       'away_team_defenceAggression', 'away_team_defenceTeamWidth',
       'away_team_buildUpPlayPositioningClass_Free_Form',
       'away_team_buildUpPlayPositioningClass_Organised',
       'away_team_chanceCreationPositioningClass_Free_Form',
       'away_team_chanceCreationPositioningClass_Organised',
       'away_team_defenceDefenderLineClass_Cover',
       'away_team_defenceDefenderLineClass_Offside_Trap',
       'home_player_1_player_api_id_x', 'home_player_1_height_x',
       'home_player_1_weight_x', 'home_player_1_overall_rating_x',
       'home_player_1_potential_x', 'home_player_1_attacking_work_rate_x',
       'home_player_1_defensive_work_rate_x', 'home_player_1_crossing_x',
       'home_player_1_finishing_x', 'home_player_1_heading_accuracy_x',
       'home_player_1_short_passing_x', 'h

In [2114]:
ml_data.columns.values[80:120]

array(['home_player_1_aggression_x', 'home_player_1_interceptions_x',
       'home_player_1_positioning_x', 'home_player_1_vision_x',
       'home_player_1_penalties_x', 'home_player_1_marking_x',
       'home_player_1_standing_tackle_x',
       'home_player_1_sliding_tackle_x', 'home_player_1_gk_diving_x',
       'home_player_1_gk_handling_x', 'home_player_1_gk_kicking_x',
       'home_player_1_gk_positioning_x', 'home_player_1_gk_reflexes_x',
       'home_player_1_preferred_foot_left_x',
       'home_player_1_preferred_foot_right_x',
       'home_player_2_player_api_id', 'home_player_2_height',
       'home_player_2_weight', 'home_player_2_overall_rating',
       'home_player_2_potential', 'home_player_2_attacking_work_rate',
       'home_player_2_defensive_work_rate', 'home_player_2_crossing',
       'home_player_2_finishing', 'home_player_2_heading_accuracy',
       'home_player_2_short_passing', 'home_player_2_volleys',
       'home_player_2_dribbling', 'home_player_2_curve',
    

We need to cut off those '_x' suffixes in some columns

In [2115]:
ml_data.columns.values[20:36]

array(['home_team_team_api_id_x', 'home_team_buildUpPlaySpeed_x',
       'home_team_buildUpPlayDribbling_x',
       'home_team_buildUpPlayPassing_x',
       'home_team_chanceCreationPassing_x',
       'home_team_chanceCreationCrossing_x',
       'home_team_chanceCreationShooting_x',
       'home_team_defencePressure_x', 'home_team_defenceAggression_x',
       'home_team_defenceTeamWidth_x',
       'home_team_buildUpPlayPositioningClass_Free_Form_x',
       'home_team_buildUpPlayPositioningClass_Organised_x',
       'home_team_chanceCreationPositioningClass_Free_Form_x',
       'home_team_chanceCreationPositioningClass_Organised_x',
       'home_team_defenceDefenderLineClass_Cover_x',
       'home_team_defenceDefenderLineClass_Offside_Trap_x'], dtype=object)

In [2116]:
for i in range(20, 36):
    ml_data.columns.values[i] = ml_data.columns.values[i][:-2]
ml_data.columns.values[20:36]

array(['home_team_team_api_id', 'home_team_buildUpPlaySpeed',
       'home_team_buildUpPlayDribbling', 'home_team_buildUpPlayPassing',
       'home_team_chanceCreationPassing',
       'home_team_chanceCreationCrossing',
       'home_team_chanceCreationShooting', 'home_team_defencePressure',
       'home_team_defenceAggression', 'home_team_defenceTeamWidth',
       'home_team_buildUpPlayPositioningClass_Free_Form',
       'home_team_buildUpPlayPositioningClass_Organised',
       'home_team_chanceCreationPositioningClass_Free_Form',
       'home_team_chanceCreationPositioningClass_Organised',
       'home_team_defenceDefenderLineClass_Cover',
       'home_team_defenceDefenderLineClass_Offside_Trap'], dtype=object)

In [2117]:
ml_data.columns.values[53:95]

array(['home_player_1_player_api_id_x', 'home_player_1_height_x',
       'home_player_1_weight_x', 'home_player_1_overall_rating_x',
       'home_player_1_potential_x', 'home_player_1_attacking_work_rate_x',
       'home_player_1_defensive_work_rate_x', 'home_player_1_crossing_x',
       'home_player_1_finishing_x', 'home_player_1_heading_accuracy_x',
       'home_player_1_short_passing_x', 'home_player_1_volleys_x',
       'home_player_1_dribbling_x', 'home_player_1_curve_x',
       'home_player_1_free_kick_accuracy_x',
       'home_player_1_long_passing_x', 'home_player_1_ball_control_x',
       'home_player_1_acceleration_x', 'home_player_1_sprint_speed_x',
       'home_player_1_agility_x', 'home_player_1_reactions_x',
       'home_player_1_balance_x', 'home_player_1_shot_power_x',
       'home_player_1_jumping_x', 'home_player_1_stamina_x',
       'home_player_1_strength_x', 'home_player_1_long_shots_x',
       'home_player_1_aggression_x', 'home_player_1_interceptions_x',
       '

In [2118]:
for i in range(53, 95):
    ml_data.columns.values[i] = ml_data.columns.values[i][:-2]
ml_data.columns.values[53:95]

array(['home_player_1_player_api_id', 'home_player_1_height',
       'home_player_1_weight', 'home_player_1_overall_rating',
       'home_player_1_potential', 'home_player_1_attacking_work_rate',
       'home_player_1_defensive_work_rate', 'home_player_1_crossing',
       'home_player_1_finishing', 'home_player_1_heading_accuracy',
       'home_player_1_short_passing', 'home_player_1_volleys',
       'home_player_1_dribbling', 'home_player_1_curve',
       'home_player_1_free_kick_accuracy', 'home_player_1_long_passing',
       'home_player_1_ball_control', 'home_player_1_acceleration',
       'home_player_1_sprint_speed', 'home_player_1_agility',
       'home_player_1_reactions', 'home_player_1_balance',
       'home_player_1_shot_power', 'home_player_1_jumping',
       'home_player_1_stamina', 'home_player_1_strength',
       'home_player_1_long_shots', 'home_player_1_aggression',
       'home_player_1_interceptions', 'home_player_1_positioning',
       'home_player_1_vision', 'home_

In [2119]:
ml_data

,season,stage,date,match_api_id,home_team_goal,away_team_goal,league_name_Belgium Jupiler League,league_name_England Premier League,league_name_France Ligue 1,league_name_Germany 1. Bundesliga,...,away_player_2_age,away_player_3_age,away_player_4_age,away_player_5_age,away_player_6_age,away_player_7_age,away_player_8_age,away_player_9_age,away_player_10_age,away_player_11_age
0,2008/2009,24,2009-02-27,493017,2,1,1,0,0,0,...,27,18,21,21,22,23,27,27,26,19
1,2010/2011,22,2010-12-30,838754,2,2,1,0,0,0,...,27,28,25,23,25,22,19,29,23,21
2,2010/2011,8,2010-09-21,838558,0,2,1,0,0,0,...,27,28,24,22,25,22,20,19,22,21
3,2010/2011,6,2010-09-12,838528,2,2,1,0,0,0,...,27,28,24,22,29,25,22,19,22,21
4,2010/2011,30,2011-03-20,838876,0,1,1,0,0,0,...,27,25,23,23,29,23,25,19,23,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008/2009,28,2009-04-17,499562,4,0,0,0,0,1,...,28,29,33,26,32,26,32,29,26,29
19608,2008/2009,12,2008-11-08,499419,3,0,0,0,0,1,...,27,32,28,25,30,30,27,31,25,28
19609,2008/2009,20,2009-02-15,499489,1,1,0,0,0,1,...,28,28,33,26,30,30,29,32,25,29
19610,2008/2009,18,2009-01-31,499478,2,0,0,0,0,1,...,28,28,33,26,30,30,27,32,27,29


In [2120]:
ml_data.columns.values[:40]

array(['season', 'stage', 'date', 'match_api_id', 'home_team_goal',
       'away_team_goal', 'league_name_Belgium Jupiler League',
       'league_name_England Premier League', 'league_name_France Ligue 1',
       'league_name_Germany 1. Bundesliga', 'league_name_Italy Serie A',
       'league_name_Netherlands Eredivisie',
       'league_name_Poland Ekstraklasa',
       'league_name_Portugal Liga ZON Sagres',
       'league_name_Scotland Premier League',
       'league_name_Spain LIGA BBVA',
       'league_name_Switzerland Super League', 'avg_odds_home_win',
       'avg_odds_draw', 'avg_odds_away_win', 'home_team_team_api_id',
       'home_team_buildUpPlaySpeed', 'home_team_buildUpPlayDribbling',
       'home_team_buildUpPlayPassing', 'home_team_chanceCreationPassing',
       'home_team_chanceCreationCrossing',
       'home_team_chanceCreationShooting', 'home_team_defencePressure',
       'home_team_defenceAggression', 'home_team_defenceTeamWidth',
       'home_team_buildUpPlayPositioni

Change 'season' and 'date' to 'season_start_year'

In [2121]:
ml_data.columns.values[0]

'season'

In [2122]:
ml_data['season']

0        2008/2009
1        2010/2011
2        2010/2011
3        2010/2011
4        2010/2011
           ...    
19607    2008/2009
19608    2008/2009
19609    2008/2009
19610    2008/2009
19611    2008/2009
Name: season, Length: 19612, dtype: object

In [2123]:
ml_data['season'].str[:4].astype('int64')

0        2008
1        2010
2        2010
3        2010
4        2010
         ... 
19607    2008
19608    2008
19609    2008
19610    2008
19611    2008
Name: season, Length: 19612, dtype: int64

In [2124]:
ml_data['season'].isnull().sum()

0

In [2125]:
ml_data['season'] = ml_data['season'].str[:4]

In [2126]:
ml_data['season'] = pd.to_numeric(ml_data['season'])

In [2127]:
ml_data['season'] = ml_data['season'].astype('int64')

In [2128]:
ml_data['season'].isnull().sum()

0

In [2129]:
# ml_data.columns.values[0] = 'season_start_year'

In [2130]:
ml_data

,season,stage,date,match_api_id,home_team_goal,away_team_goal,league_name_Belgium Jupiler League,league_name_England Premier League,league_name_France Ligue 1,league_name_Germany 1. Bundesliga,...,away_player_2_age,away_player_3_age,away_player_4_age,away_player_5_age,away_player_6_age,away_player_7_age,away_player_8_age,away_player_9_age,away_player_10_age,away_player_11_age
0,2008,24,2009-02-27,493017,2,1,1,0,0,0,...,27,18,21,21,22,23,27,27,26,19
1,2010,22,2010-12-30,838754,2,2,1,0,0,0,...,27,28,25,23,25,22,19,29,23,21
2,2010,8,2010-09-21,838558,0,2,1,0,0,0,...,27,28,24,22,25,22,20,19,22,21
3,2010,6,2010-09-12,838528,2,2,1,0,0,0,...,27,28,24,22,29,25,22,19,22,21
4,2010,30,2011-03-20,838876,0,1,1,0,0,0,...,27,25,23,23,29,23,25,19,23,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008,28,2009-04-17,499562,4,0,0,0,0,1,...,28,29,33,26,32,26,32,29,26,29
19608,2008,12,2008-11-08,499419,3,0,0,0,0,1,...,27,32,28,25,30,30,27,31,25,28
19609,2008,20,2009-02-15,499489,1,1,0,0,0,1,...,28,28,33,26,30,30,29,32,25,29
19610,2008,18,2009-01-31,499478,2,0,0,0,0,1,...,28,28,33,26,30,30,27,32,27,29


In [2131]:
# ml_data['season_start_year'].isnull().sum()

In [2132]:
ml_data.columns.values[2]

'date'

In [2133]:
ml_data.drop('date', axis=1, inplace=True)

In [2134]:
ml_data.columns.values[52:99]

array(['home_player_1_player_api_id', 'home_player_1_height',
       'home_player_1_weight', 'home_player_1_overall_rating',
       'home_player_1_potential', 'home_player_1_attacking_work_rate',
       'home_player_1_defensive_work_rate', 'home_player_1_crossing',
       'home_player_1_finishing', 'home_player_1_heading_accuracy',
       'home_player_1_short_passing', 'home_player_1_volleys',
       'home_player_1_dribbling', 'home_player_1_curve',
       'home_player_1_free_kick_accuracy', 'home_player_1_long_passing',
       'home_player_1_ball_control', 'home_player_1_acceleration',
       'home_player_1_sprint_speed', 'home_player_1_agility',
       'home_player_1_reactions', 'home_player_1_balance',
       'home_player_1_shot_power', 'home_player_1_jumping',
       'home_player_1_stamina', 'home_player_1_strength',
       'home_player_1_long_shots', 'home_player_1_aggression',
       'home_player_1_interceptions', 'home_player_1_positioning',
       'home_player_1_vision', 'home_

Remove match, teams and players ids (not needed for training, train just based on tactics and skills)

In [2135]:
ml_data.columns.values[2]

'match_api_id'

In [2136]:
ml_data.drop('match_api_id', axis=1, inplace=True)

In [2137]:
ml_data.columns.values[18]

'home_team_team_api_id'

In [2138]:
ml_data.drop('home_team_team_api_id', axis=1, inplace=True)

In [2139]:
ml_data.columns.values[33]

'away_team_team_api_id'

In [2140]:
ml_data.drop('away_team_team_api_id', axis=1, inplace=True)

In [2141]:
ml_data.columns.values[33]

'away_team_team_long_name'

In [2142]:
ml_data.drop('away_team_team_long_name', axis=1, inplace=True)

In [2143]:
ml_data.columns.values[-22]

'home_player_1_age_x'

In [2144]:
ml_data.columns.values[-22] = 'home_player_1_age'

In [2145]:
ml_data.columns.values[-23:]

array(['away_player_11_preferred_foot_right', 'home_player_1_age',
       'home_player_2_age', 'home_player_3_age', 'home_player_4_age',
       'home_player_5_age', 'home_player_6_age', 'home_player_7_age',
       'home_player_8_age', 'home_player_9_age', 'home_player_10_age',
       'home_player_11_age', 'away_player_1_age', 'away_player_2_age',
       'away_player_3_age', 'away_player_4_age', 'away_player_5_age',
       'away_player_6_age', 'away_player_7_age', 'away_player_8_age',
       'away_player_9_age', 'away_player_10_age', 'away_player_11_age'],
      dtype=object)

In [2146]:
player_ids = ml_data.columns.values[48::42][:-1]
player_ids

array(['home_player_1_player_api_id', 'home_player_2_player_api_id',
       'home_player_3_player_api_id', 'home_player_4_player_api_id',
       'home_player_5_player_api_id', 'home_player_6_player_api_id',
       'home_player_7_player_api_id', 'home_player_8_player_api_id',
       'home_player_9_player_api_id', 'home_player_10_player_api_id',
       'home_player_11_player_api_id', 'away_player_1_player_api_id',
       'away_player_2_player_api_id', 'away_player_3_player_api_id',
       'away_player_4_player_api_id', 'away_player_5_player_api_id',
       'away_player_6_player_api_id', 'away_player_7_player_api_id',
       'away_player_8_player_api_id', 'away_player_9_player_api_id',
       'away_player_10_player_api_id', 'away_player_11_player_api_id'],
      dtype=object)

In [2147]:
ml_data.drop(player_ids, axis=1, inplace=True)

Remove gk attributes for out-field players (those attributes will not have impact on match). This decreases the total number of attributes by 100. The only GKs are **home_player_1** and **away_player_1**

In [2148]:
players_gk_divings = ml_data.columns.values[82::41]
players_gk_divings

array(['home_player_1_gk_diving', 'home_player_2_gk_diving',
       'home_player_3_gk_diving', 'home_player_4_gk_diving',
       'home_player_5_gk_diving', 'home_player_6_gk_diving',
       'home_player_7_gk_diving', 'home_player_8_gk_diving',
       'home_player_9_gk_diving', 'home_player_10_gk_diving',
       'home_player_11_gk_diving', 'away_player_1_gk_diving',
       'away_player_2_gk_diving', 'away_player_3_gk_diving',
       'away_player_4_gk_diving', 'away_player_5_gk_diving',
       'away_player_6_gk_diving', 'away_player_7_gk_diving',
       'away_player_8_gk_diving', 'away_player_9_gk_diving',
       'away_player_10_gk_diving', 'away_player_11_gk_diving'],
      dtype=object)

In [2149]:
players_gk_divings = np.delete(players_gk_divings, 0)
players_gk_divings = np.delete(players_gk_divings, 10)
players_gk_divings

array(['home_player_2_gk_diving', 'home_player_3_gk_diving',
       'home_player_4_gk_diving', 'home_player_5_gk_diving',
       'home_player_6_gk_diving', 'home_player_7_gk_diving',
       'home_player_8_gk_diving', 'home_player_9_gk_diving',
       'home_player_10_gk_diving', 'home_player_11_gk_diving',
       'away_player_2_gk_diving', 'away_player_3_gk_diving',
       'away_player_4_gk_diving', 'away_player_5_gk_diving',
       'away_player_6_gk_diving', 'away_player_7_gk_diving',
       'away_player_8_gk_diving', 'away_player_9_gk_diving',
       'away_player_10_gk_diving', 'away_player_11_gk_diving'],
      dtype=object)

In [2150]:
players_gk_handlings = ml_data.columns.values[83::41]
players_gk_handlings

array(['home_player_1_gk_handling', 'home_player_2_gk_handling',
       'home_player_3_gk_handling', 'home_player_4_gk_handling',
       'home_player_5_gk_handling', 'home_player_6_gk_handling',
       'home_player_7_gk_handling', 'home_player_8_gk_handling',
       'home_player_9_gk_handling', 'home_player_10_gk_handling',
       'home_player_11_gk_handling', 'away_player_1_gk_handling',
       'away_player_2_gk_handling', 'away_player_3_gk_handling',
       'away_player_4_gk_handling', 'away_player_5_gk_handling',
       'away_player_6_gk_handling', 'away_player_7_gk_handling',
       'away_player_8_gk_handling', 'away_player_9_gk_handling',
       'away_player_10_gk_handling', 'away_player_11_gk_handling'],
      dtype=object)

In [2151]:
players_gk_handlings = np.delete(players_gk_handlings, 0)
players_gk_handlings = np.delete(players_gk_handlings, 10)
players_gk_handlings

array(['home_player_2_gk_handling', 'home_player_3_gk_handling',
       'home_player_4_gk_handling', 'home_player_5_gk_handling',
       'home_player_6_gk_handling', 'home_player_7_gk_handling',
       'home_player_8_gk_handling', 'home_player_9_gk_handling',
       'home_player_10_gk_handling', 'home_player_11_gk_handling',
       'away_player_2_gk_handling', 'away_player_3_gk_handling',
       'away_player_4_gk_handling', 'away_player_5_gk_handling',
       'away_player_6_gk_handling', 'away_player_7_gk_handling',
       'away_player_8_gk_handling', 'away_player_9_gk_handling',
       'away_player_10_gk_handling', 'away_player_11_gk_handling'],
      dtype=object)

In [2152]:
players_gk_kickings = ml_data.columns.values[84::41]
players_gk_kickings

array(['home_player_1_gk_kicking', 'home_player_2_gk_kicking',
       'home_player_3_gk_kicking', 'home_player_4_gk_kicking',
       'home_player_5_gk_kicking', 'home_player_6_gk_kicking',
       'home_player_7_gk_kicking', 'home_player_8_gk_kicking',
       'home_player_9_gk_kicking', 'home_player_10_gk_kicking',
       'home_player_11_gk_kicking', 'away_player_1_gk_kicking',
       'away_player_2_gk_kicking', 'away_player_3_gk_kicking',
       'away_player_4_gk_kicking', 'away_player_5_gk_kicking',
       'away_player_6_gk_kicking', 'away_player_7_gk_kicking',
       'away_player_8_gk_kicking', 'away_player_9_gk_kicking',
       'away_player_10_gk_kicking', 'away_player_11_gk_kicking'],
      dtype=object)

In [2153]:
players_gk_kickings = np.delete(players_gk_kickings, 0)
players_gk_kickings = np.delete(players_gk_kickings, 10)
players_gk_kickings

array(['home_player_2_gk_kicking', 'home_player_3_gk_kicking',
       'home_player_4_gk_kicking', 'home_player_5_gk_kicking',
       'home_player_6_gk_kicking', 'home_player_7_gk_kicking',
       'home_player_8_gk_kicking', 'home_player_9_gk_kicking',
       'home_player_10_gk_kicking', 'home_player_11_gk_kicking',
       'away_player_2_gk_kicking', 'away_player_3_gk_kicking',
       'away_player_4_gk_kicking', 'away_player_5_gk_kicking',
       'away_player_6_gk_kicking', 'away_player_7_gk_kicking',
       'away_player_8_gk_kicking', 'away_player_9_gk_kicking',
       'away_player_10_gk_kicking', 'away_player_11_gk_kicking'],
      dtype=object)

In [2154]:
players_gk_positionings = ml_data.columns.values[85::41]
players_gk_positionings

array(['home_player_1_gk_positioning', 'home_player_2_gk_positioning',
       'home_player_3_gk_positioning', 'home_player_4_gk_positioning',
       'home_player_5_gk_positioning', 'home_player_6_gk_positioning',
       'home_player_7_gk_positioning', 'home_player_8_gk_positioning',
       'home_player_9_gk_positioning', 'home_player_10_gk_positioning',
       'home_player_11_gk_positioning', 'away_player_1_gk_positioning',
       'away_player_2_gk_positioning', 'away_player_3_gk_positioning',
       'away_player_4_gk_positioning', 'away_player_5_gk_positioning',
       'away_player_6_gk_positioning', 'away_player_7_gk_positioning',
       'away_player_8_gk_positioning', 'away_player_9_gk_positioning',
       'away_player_10_gk_positioning', 'away_player_11_gk_positioning'],
      dtype=object)

In [2155]:
players_gk_positionings = np.delete(players_gk_positionings, 0)
players_gk_positionings = np.delete(players_gk_positionings, 10)
players_gk_positionings

array(['home_player_2_gk_positioning', 'home_player_3_gk_positioning',
       'home_player_4_gk_positioning', 'home_player_5_gk_positioning',
       'home_player_6_gk_positioning', 'home_player_7_gk_positioning',
       'home_player_8_gk_positioning', 'home_player_9_gk_positioning',
       'home_player_10_gk_positioning', 'home_player_11_gk_positioning',
       'away_player_2_gk_positioning', 'away_player_3_gk_positioning',
       'away_player_4_gk_positioning', 'away_player_5_gk_positioning',
       'away_player_6_gk_positioning', 'away_player_7_gk_positioning',
       'away_player_8_gk_positioning', 'away_player_9_gk_positioning',
       'away_player_10_gk_positioning', 'away_player_11_gk_positioning'],
      dtype=object)

In [2156]:
players_gk_reflexes = ml_data.columns.values[86::41]
players_gk_reflexes

array(['home_player_1_gk_reflexes', 'home_player_2_gk_reflexes',
       'home_player_3_gk_reflexes', 'home_player_4_gk_reflexes',
       'home_player_5_gk_reflexes', 'home_player_6_gk_reflexes',
       'home_player_7_gk_reflexes', 'home_player_8_gk_reflexes',
       'home_player_9_gk_reflexes', 'home_player_10_gk_reflexes',
       'home_player_11_gk_reflexes', 'away_player_1_gk_reflexes',
       'away_player_2_gk_reflexes', 'away_player_3_gk_reflexes',
       'away_player_4_gk_reflexes', 'away_player_5_gk_reflexes',
       'away_player_6_gk_reflexes', 'away_player_7_gk_reflexes',
       'away_player_8_gk_reflexes', 'away_player_9_gk_reflexes',
       'away_player_10_gk_reflexes', 'away_player_11_gk_reflexes'],
      dtype=object)

In [2157]:
players_gk_reflexes = np.delete(players_gk_reflexes, 0)
players_gk_reflexes = np.delete(players_gk_reflexes, 10)
players_gk_reflexes

array(['home_player_2_gk_reflexes', 'home_player_3_gk_reflexes',
       'home_player_4_gk_reflexes', 'home_player_5_gk_reflexes',
       'home_player_6_gk_reflexes', 'home_player_7_gk_reflexes',
       'home_player_8_gk_reflexes', 'home_player_9_gk_reflexes',
       'home_player_10_gk_reflexes', 'home_player_11_gk_reflexes',
       'away_player_2_gk_reflexes', 'away_player_3_gk_reflexes',
       'away_player_4_gk_reflexes', 'away_player_5_gk_reflexes',
       'away_player_6_gk_reflexes', 'away_player_7_gk_reflexes',
       'away_player_8_gk_reflexes', 'away_player_9_gk_reflexes',
       'away_player_10_gk_reflexes', 'away_player_11_gk_reflexes'],
      dtype=object)

In [2158]:
ml_data.drop(players_gk_divings, axis=1, inplace=True)
ml_data.drop(players_gk_handlings, axis=1, inplace=True)
ml_data.drop(players_gk_kickings, axis=1, inplace=True)
ml_data.drop(players_gk_positionings, axis=1, inplace=True)
ml_data.drop(players_gk_reflexes, axis=1, inplace=True)

In [2159]:
ml_data

,season,stage,home_team_goal,away_team_goal,league_name_Belgium Jupiler League,league_name_England Premier League,league_name_France Ligue 1,league_name_Germany 1. Bundesliga,league_name_Italy Serie A,league_name_Netherlands Eredivisie,...,away_player_2_age,away_player_3_age,away_player_4_age,away_player_5_age,away_player_6_age,away_player_7_age,away_player_8_age,away_player_9_age,away_player_10_age,away_player_11_age
0,2008,24,2,1,1,0,0,0,0,0,...,27,18,21,21,22,23,27,27,26,19
1,2010,22,2,2,1,0,0,0,0,0,...,27,28,25,23,25,22,19,29,23,21
2,2010,8,0,2,1,0,0,0,0,0,...,27,28,24,22,25,22,20,19,22,21
3,2010,6,2,2,1,0,0,0,0,0,...,27,28,24,22,29,25,22,19,22,21
4,2010,30,0,1,1,0,0,0,0,0,...,27,25,23,23,29,23,25,19,23,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,2008,28,4,0,0,0,0,1,0,0,...,28,29,33,26,32,26,32,29,26,29
19608,2008,12,3,0,0,0,0,1,0,0,...,27,32,28,25,30,30,27,31,25,28
19609,2008,20,1,1,0,0,0,1,0,0,...,28,28,33,26,30,30,29,32,25,29
19610,2008,18,2,0,0,0,0,1,0,0,...,28,28,33,26,30,30,27,32,27,29


In [2160]:
ml_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19612 entries, 0 to 19611
Columns: 872 entries, season to away_player_11_age
dtypes: float64(772), int64(47), uint8(53)
memory usage: 123.7 MB


In [2161]:
ml_data.describe()

,season,stage,home_team_goal,away_team_goal,league_name_Belgium Jupiler League,league_name_England Premier League,league_name_France Ligue 1,league_name_Germany 1. Bundesliga,league_name_Italy Serie A,league_name_Netherlands Eredivisie,...,away_player_2_age,away_player_3_age,away_player_4_age,away_player_5_age,away_player_6_age,away_player_7_age,away_player_8_age,away_player_9_age,away_player_10_age,away_player_11_age
count,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,...,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000,19612.000000
mean,2011.709005,18.650520,1.554609,1.171477,0.060983,0.151030,0.145880,0.121099,0.139914,0.103610,...,26.057924,26.929482,26.748623,25.825668,26.062819,26.254130,25.536253,25.316235,25.596574,25.703447
std,2.212908,10.513613,1.303719,1.150150,0.239305,0.358087,0.352995,0.326251,0.346907,0.304762,...,3.808454,4.053499,4.072170,3.821464,3.947676,3.974213,3.783744,3.818846,3.818628,3.828545
min,2008.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,17.000000,17.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
25%,2010.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.000000,24.000000,24.000000,23.000000,23.000000,23.000000,23.000000,22.000000,23.000000,23.000000
50%,2012.000000,19.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,26.000000,27.000000,26.000000,26.000000,26.000000,26.000000,25.000000,25.000000,25.000000,25.000000
75%,2014.000000,28.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,29.000000,30.000000,30.000000,28.000000,29.000000,29.000000,28.000000,28.000000,28.000000,28.000000
max,2015.000000,38.000000,10.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,40.000000,40.000000,41.000000,40.000000,40.000000,40.000000,39.000000,40.000000,39.000000,38.000000


In [2162]:
ml_data.to_pickle("data/ml_data.pkl")